# 4 - Mixed Multi Input Models

## Group 1

Mikołaj Jarząbkowski | 20230529 <br>
Rodrigo Silva | 20230536 <br>
Nicolau Dulea | 20230544 <br>
Joana Gonçalves | 20230977 <br>
Tiago Fernandes | 20230988 <br>

In [1]:
from numpy.random import seed
seed(42)

import math
#! pip install tensorflow 
import tensorflow 
from tensorflow.random import set_seed
tensorflow.random.set_seed(42)
# importing needed libraries
#%tensorflow_version 1.x
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import os, shutil
import io
import seaborn as sns
import requests
import urllib
import bs4
import re
from PIL import Image
from io import BytesIO

# keras
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout, Input,concatenate,RandomRotation, RandomZoom,Rescaling
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import optimizers,models,callbacks , layers
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.applications import VGG19,ResNet152V2



# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import f1_score,confusion_matrix,classification_report, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight

# matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [304]:
df = pd.read_csv("fitzpatrick17k.csv")
df

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
0,5e82a45bc5d78bd24ae9202d194423f8,3,3,drug induced pigmentary changes,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmminoc...
1,fa2911a9b13b6f8af79cb700937cc14f,1,1,photodermatoses,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicpphoto...
2,d2bac3c9e4499032ca8e9b07c7d3bc40,2,3,dermatofibroma,benign dermal,benign,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicdderma...
3,0a94359e7eaacd7178e06b2823777789,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
4,a39ec3b1f22c08a421fa20535e037bba,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
...,...,...,...,...,...,...,...,...,...
16572,cd90e491ddaa92f0f4eb07f73aa09f64,5,5,scleroderma,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=4529,httpwwwatlasdermatologicocombrimgimageId4529.jpg
16573,5f1ed6de6a9110d7dc580a6a0312af63,4,5,pityriasis rosea,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=5630,httpwwwatlasdermatologicocombrimgimageId5630.jpg
16574,f198aaf1f0550c2464b285454d34926e,3,3,myiasis,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=4609,httpwwwatlasdermatologicocombrimgimageId4609.jpg
16575,6214de2e915835014235a1839cbc5938,-1,5,vitiligo,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=7930,httpwwwatlasdermatologicocombrimgimageId7930.jpg


In [305]:
# Dropping out the rows in column "url" that have NaN values, and saving them for maybe a usage in future
url_droped = df[df["url"].isna()]
df.dropna(subset = ["url"], inplace = True)

In [306]:
# Dropping out the "md5hash", does look necessary
df.drop(columns = "md5hash",inplace = True)

In [307]:
# Dropping out all bar Diagnostic since we dont like it because or its Wrongly labelled or Potentially and we  dont want to do a model with potentially 
df = df[~df['qc'].isin(["5 Potentially", "2 Characteristic", "3 Wrongly labelled", "4 Other"])]

In [308]:
# Dropping out "url_alphanum", it has no use for the training part
df.drop(columns = "url_alphanum", inplace = True)

In [309]:
# Encode categorical features
categorical_feats = ['nine_partition_label', 'three_partition_label']
df= pd.get_dummies(df, columns=categorical_feats)

df.replace(False, 0, inplace=True)
df.replace(True, 1, inplace=True)

### Download Images

In [313]:
df.head()

,fitzpatrick_scale,fitzpatrick_centaur,label,qc,url,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,3,3,drug induced pigmentary changes,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
1,1,1,photodermatoses,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
2,2,3,dermatofibroma,NaN,https://www.dermaamin.com/site/images/clinical...,1,0,0,0,0,0,0,0,0,1,0,0
3,1,1,psoriasis,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
4,1,1,psoriasis,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1


In [315]:
def download_images(df):
    """
    This function downloads images specified in the 'url' column of the given DataFrame, saves them in a folder called 'images', and updates the DataFrame with new columns 'image_name' and 'image_path'.
    """
    # Create folder if it doesn't exist
    os.makedirs("images_full", exist_ok=True)
    
    # Dictionary to store the count of each label
    label_counts = {}
    
    # List to store the rows to drop
    rows_to_drop = []
    
    # Function to sanitize the filename
    def sanitize_filename(filename):
        return re.sub(r'[^\w\-_.]', '_', filename)
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        label = row['label']
        image_url = row['url']
        file_name = sanitize_filename(image_url.split("/")[-1])
        
        # Increment the count for the label
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 0
        
        # Construct the filename with label and count
        file_name = f"{label}_{label_counts[label]}"
        
        # Define the path to save the image
        save_path = os.path.join("images_full", f"{file_name}.jpg")
        
        # Check if the image file already exists
        if not os.path.exists(save_path):
            # Define headers to mimic a browser request
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
            }
            
            # Send a GET request to the URL to retrieve the image data
            response = requests.get(image_url, headers=headers)
            
            # Check if the request was successful
            if response.status_code == 200:
                # Open the image using PIL (Python Imaging Library)
                image = Image.open(BytesIO(response.content))
                
                # Save the image as JPG
                image.save(save_path)
                print(f"Image '{file_name}.jpg' downloaded successfully.")
                
                # Update DataFrame with image name and path
                df.at[index, 'image_name'] = file_name
                df.at[index, 'image_path'] = save_path
            else:
                print(f"Failed to download image for label '{label}' from {image_url}")
                # Mark the row to drop
                rows_to_drop.append(index)
        else:
            print(f"Image '{file_name}.jpg' already exists.")
            # Update DataFrame with image name and path
            df.at[index, 'image_name'] = file_name
            df.at[index, 'image_path'] = save_path
    
    # Drop rows with unsuccessful downloads
    df = df.drop(rows_to_drop)
    
    # Reindex DataFrame starting from 0
    df.reset_index(drop=True, inplace=True)
    
    return df

# Call the function with your DataFrame
df = download_images(df)

Image 'drug induced pigmentary changes_0.jpg' already exists.
Image 'photodermatoses_0.jpg' already exists.
Image 'dermatofibroma_0.jpg' already exists.
Image 'psoriasis_0.jpg' already exists.
Image 'psoriasis_1.jpg' already exists.
Image 'kaposi sarcoma_0.jpg' already exists.
Image 'neutrophilic dermatoses_0.jpg' already exists.
Image 'granuloma annulare_0.jpg' already exists.
Image 'nematode infection_0.jpg' already exists.
Image 'allergic contact dermatitis_0.jpg' already exists.
Image 'necrobiosis lipoidica_0.jpg' already exists.
Image 'neutrophilic dermatoses_1.jpg' already exists.
Image 'hidradenitis_0.jpg' already exists.
Image 'melanoma_0.jpg' already exists.
Image 'acne vulgaris_0.jpg' already exists.
Image 'necrobiosis lipoidica_1.jpg' already exists.
Image 'sarcoidosis_0.jpg' already exists.
Image 'xeroderma pigmentosum_0.jpg' already exists.
Image 'melanoma_1.jpg' already exists.
Image 'dermatofibroma_1.jpg' already exists.
Image 'actinic keratosis_0.jpg' already exists.
Im

Image 'lupus erythematosus_3.jpg' already exists.
Image 'stasis edema_1.jpg' already exists.
Image 'stevens johnson syndrome_3.jpg' already exists.
Image 'aplasia cutis_1.jpg' already exists.
Image 'acne_3.jpg' already exists.
Image 'allergic contact dermatitis_6.jpg' already exists.
Image 'pyogenic granuloma_0.jpg' already exists.
Image 'langerhans cell histiocytosis_0.jpg' already exists.
Image 'neutrophilic dermatoses_10.jpg' already exists.
Image 'psoriasis_12.jpg' already exists.
Image 'lyme disease_3.jpg' already exists.
Image 'lymphangioma_1.jpg' already exists.
Image 'port wine stain_0.jpg' already exists.
Image 'malignant melanoma_0.jpg' already exists.
Image 'folliculitis_3.jpg' already exists.
Image 'acne vulgaris_4.jpg' already exists.
Image 'psoriasis_13.jpg' already exists.
Image 'porphyria_3.jpg' already exists.
Image 'actinic keratosis_3.jpg' already exists.
Image 'neutrophilic dermatoses_11.jpg' already exists.
Image 'kaposi sarcoma_2.jpg' already exists.
Image 'rhinop

Image 'basal cell carcinoma_10.jpg' already exists.
Image 'malignant melanoma_5.jpg' already exists.
Image 'dariers disease_6.jpg' already exists.
Image 'dariers disease_7.jpg' already exists.
Image 'lichen planus_6.jpg' already exists.
Image 'granuloma annulare_7.jpg' already exists.
Image 'myiasis_0.jpg' already exists.
Image 'papilomatosis confluentes and reticulate_4.jpg' already exists.
Image 'neutrophilic dermatoses_16.jpg' already exists.
Image 'granuloma annulare_8.jpg' already exists.
Image 'seborrheic dermatitis_5.jpg' already exists.
Image 'urticaria pigmentosa_2.jpg' already exists.
Image 'lupus erythematosus_8.jpg' already exists.
Image 'dermatofibroma_5.jpg' already exists.
Image 'perioral dermatitis_2.jpg' already exists.
Image 'allergic contact dermatitis_12.jpg' already exists.
Image 'acne vulgaris_12.jpg' already exists.
Image 'acne_7.jpg' already exists.
Image 'rosacea_2.jpg' already exists.
Image 'dyshidrotic eczema_4.jpg' already exists.
Image 'squamous cell carcin

Image 'psoriasis_30.jpg' already exists.
Image 'paronychia_5.jpg' already exists.
Image 'cheilitis_7.jpg' already exists.
Image 'squamous cell carcinoma_23.jpg' already exists.
Image 'sarcoidosis_12.jpg' already exists.
Image 'photodermatoses_15.jpg' already exists.
Image 'lymphangioma_4.jpg' already exists.
Image 'dyshidrotic eczema_6.jpg' already exists.
Image 'basal cell carcinoma_14.jpg' already exists.
Image 'behcets disease_4.jpg' already exists.
Image 'lichen planus_15.jpg' already exists.
Image 'epidermal nevus_2.jpg' already exists.
Image 'becker nevus_4.jpg' already exists.
Image 'sarcoidosis_13.jpg' already exists.
Image 'photodermatoses_16.jpg' already exists.
Image 'acne vulgaris_23.jpg' already exists.
Image 'squamous cell carcinoma_24.jpg' already exists.
Image 'lupus erythematosus_17.jpg' already exists.
Image 'tuberous sclerosis_4.jpg' already exists.
Image 'folliculitis_8.jpg' already exists.
Image 'seborrheic keratosis_0.jpg' already exists.
Image 'acne vulgaris_24.j

Image 'drug eruption_12.jpg' already exists.
Image 'acne_20.jpg' already exists.
Image 'folliculitis_12.jpg' already exists.
Image 'necrobiosis lipoidica_9.jpg' already exists.
Image 'acrodermatitis enteropathica_3.jpg' already exists.
Image 'psoriasis_38.jpg' already exists.
Image 'lupus erythematosus_28.jpg' already exists.
Image 'pyogenic granuloma_5.jpg' already exists.
Image 'psoriasis_39.jpg' already exists.
Image 'acne vulgaris_31.jpg' already exists.
Image 'lupus erythematosus_29.jpg' already exists.
Image 'psoriasis_40.jpg' already exists.
Image 'rosacea_7.jpg' already exists.
Image 'sarcoidosis_22.jpg' already exists.
Image 'nevocytic nevus_7.jpg' already exists.
Image 'acquired autoimmune bullous diseaseherpes gestationis_3.jpg' already exists.
Image 'superficial spreading melanoma ssm_11.jpg' already exists.
Image 'nematode infection_10.jpg' already exists.
Image 'allergic contact dermatitis_29.jpg' already exists.
Image 'epidermal nevus_5.jpg' already exists.
Image 'kaposi

...

Image 'perioral dermatitis_58.jpg' downloaded successfully.
Image 'neurofibromatosis_169.jpg' already exists.
Image 'kaposi sarcoma_144.jpg' already exists.
Image 'melanoma_231.jpg' already exists.
Image 'folliculitis_319.jpg' already exists.
Image 'neutrophilic dermatoses_335.jpg' already exists.
Image 'scleroderma_298.jpg' already exists.
Image 'erythema elevatum diutinum_51.jpg' already exists.
Image 'pityriasis rubra pilaris_234.jpg' already exists.
Image 'porokeratosis actinic_156.jpg' already exists.
Image 'squamous cell carcinoma_534.jpg' already exists.
Image 'folliculitis_320.jpg' already exists.
Image 'erythema annulare centrifigum_85.jpg' already exists.
Image 'papilomatosis confluentes and reticulate_126.jpg' already exists.
Image 'squamous cell carcinoma_535.jpg' already exists.
Image 'mycosis fungoides_162.jpg' already exists.
Image 'neurofibromatosis_170.jpg' already exists.
Image 'neurofibromatosis_171.jpg' already exists.
Image 'cheilitis_102.jpg' already exists.
Image

Image 'scabies_310.jpg' already exists.
Image 'pityriasis rubra pilaris_241.jpg' already exists.
Image 'hailey hailey disease_160.jpg' already exists.
Image 'psoriasis_626.jpg' already exists.
Image 'lymphangioma_96.jpg' already exists.
Image 'acanthosis nigricans_85.jpg' already exists.
Image 'allergic contact dermatitis_408.jpg' already exists.
Image 'prurigo nodularis_161.jpg' already exists.
Image 'papilomatosis confluentes and reticulate_133.jpg' already exists.
Image 'sarcoidosis_333.jpg' already exists.
Image 'actinic keratosis_169.jpg' already exists.
Image 'nematode infection_233.jpg' already exists.
Image 'basal cell carcinoma_424.jpg' already exists.
Image 'hailey hailey disease_161.jpg' already exists.
Image 'lichen planus_439.jpg' already exists.
Image 'dariers disease_136.jpg' already exists.
Image 'stasis edema_67.jpg' already exists.
Image 'myiasis_55.jpg' already exists.
Image 'porphyria_121.jpg' already exists.
Image 'neutrophilic dermatoses_340.jpg' already exists.
I

Image 'lupus subacute_105.jpg' downloaded successfully.
Image 'melanoma_240.jpg' already exists.
Image 'folliculitis_326.jpg' already exists.
Image 'pityriasis rubra pilaris_247.jpg' already exists.
Image 'squamous cell carcinoma_546.jpg' already exists.
Image 'lupus erythematosus_390.jpg' already exists.
Image 'melanoma_241.jpg' already exists.
Image 'lichen simplex_59.jpg' already exists.
Image 'hailey hailey disease_165.jpg' already exists.
Image 'tuberous sclerosis_132.jpg' already exists.
Image 'syringoma_120.jpg' already exists.
Image 'scabies_320.jpg' already exists.
Image 'tuberous sclerosis_133.jpg' already exists.
Image 'porokeratosis actinic_166.jpg' already exists.
Image 'allergic contact dermatitis_412.jpg' already exists.
Image 'neutrophilic dermatoses_343.jpg' already exists.
Image 'rhinophyma_89.jpg' already exists.
Image 'lichen planus_450.jpg' already exists.
Image 'squamous cell carcinoma_547.jpg' already exists.
Image 'pityriasis rubra pilaris_248.jpg' already exist

Image 'neurofibromatosis_179.jpg' already exists.
Image 'scabies_325.jpg' already exists.
Image 'neutrophilic dermatoses_344.jpg' already exists.
Image 'basal cell carcinoma_448.jpg' already exists.
Image 'acanthosis nigricans_88.jpg' already exists.
Image 'syringoma_123.jpg' already exists.
Image 'pityriasis rubra pilaris_257.jpg' already exists.
Image 'basal cell carcinoma_449.jpg' already exists.
Image 'psoriasis_636.jpg' already exists.
Image 'pityriasis rubra pilaris_258.jpg' already exists.
Image 'allergic contact dermatitis_414.jpg' already exists.
Image 'erythema multiforme_231.jpg' already exists.
Image 'folliculitis_331.jpg' already exists.
Image 'pityriasis rosea_181.jpg' already exists.
Image 'pediculosis lids_141.jpg' already exists.
Image 'lupus erythematosus_393.jpg' already exists.
Image 'hailey hailey disease_171.jpg' already exists.
Image 'melanoma_247.jpg' already exists.
Image 'pilar cyst_91.jpg' already exists.
Image 'lichen planus_462.jpg' already exists.
Image 'l

Image 'nematode infection_252.jpg' already exists.
Image 'melanoma_253.jpg' already exists.
Image 'lichen planus_480.jpg' already exists.
Image 'erythema nodosum_80.jpg' already exists.
Image 'neurofibromatosis_183.jpg' already exists.
Image 'tungiasis_146.jpg' already exists.
Image 'lupus erythematosus_398.jpg' already exists.
Image 'neurofibromatosis_184.jpg' already exists.
Image 'neurotic excoriations_71.jpg' already exists.
Image 'urticaria_148.jpg' already exists.
Image 'fixed eruptions_122.jpg' already exists.
Image 'hailey hailey disease_173.jpg' already exists.
Image 'porokeratosis actinic_179.jpg' already exists.
Image 'acrodermatitis enteropathica_89.jpg' already exists.
Image 'acne_180.jpg' already exists.
Image 'papilomatosis confluentes and reticulate_146.jpg' already exists.
Image 'papilomatosis confluentes and reticulate_147.jpg' already exists.
Image 'neutrophilic dermatoses_353.jpg' already exists.
Image 'fixed eruptions_123.jpg' already exists.
Image 'folliculitis_33

In [316]:
# Reorder columns
df = df[['image_name', 'image_path', 'label'] + [col for col in df.columns if col not in ['image_name', 'image_path', 'label']]]

In [317]:
# Salvar as informações em um arquivo CSV
df.to_csv('fitzpatrick17k_2', index=False)

## CNN with Images + Features

**CNN + nine_partition_label**

In [48]:
df = pd.read_csv('fitzpatrick17k_3')
df

,image_name,image_path,label,fitzpatrick_scale,fitzpatrick_centaur,qc,url,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,drug induced pigmentary changes_0,images_full\drug induced pigmentary changes_0.jpg,drug induced pigmentary changes,3,3,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
1,photodermatoses_0,images_full\photodermatoses_0.jpg,photodermatoses,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
2,dermatofibroma_0,images_full\dermatofibroma_0.jpg,dermatofibroma,2,3,NaN,https://www.dermaamin.com/site/images/clinical...,1,0,0,0,0,0,0,0,0,1,0,0
3,psoriasis_0,images_full\psoriasis_0.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
4,psoriasis_1,images_full\psoriasis_1.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,scleroderma_306,images_full\scleroderma_306.jpg,scleroderma,5,5,NaN,http://atlasdermatologico.com.br/img?imageId=4529,0,0,0,0,1,0,0,0,0,0,0,1
16366,pityriasis rosea_190,images_full\pityriasis rosea_190.jpg,pityriasis rosea,4,5,NaN,http://atlasdermatologico.com.br/img?imageId=5630,0,0,0,0,1,0,0,0,0,0,0,1
16367,myiasis_66,images_full\myiasis_66.jpg,myiasis,3,3,NaN,http://atlasdermatologico.com.br/img?imageId=4609,0,0,0,0,1,0,0,0,0,0,0,1
16368,vitiligo_164,images_full\vitiligo_164.jpg,vitiligo,-1,5,NaN,http://atlasdermatologico.com.br/img?imageId=7930,0,0,0,0,1,0,0,0,0,0,0,1


In [49]:
df = df.drop(['fitzpatrick_centaur', 'fitzpatrick_scale','qc', 'url', 'three_partition_label_benign', 'three_partition_label_malignant', 'three_partition_label_non-neoplastic'], axis=1)

In [50]:
# Replace 'images' with 'images_resized' in the 'image_path' column
df['image_path'] = df['image_path'].str.replace('images_full', 'images_resized_full')

In [51]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Visualize the unique and encoded labels
print("Unique labels:", label_encoder.classes_)
print("Encoded labels:", df['label'].unique())

Unique labels: ['acanthosis nigricans' 'acne' 'acne vulgaris'
 'acquired autoimmune bullous diseaseherpes gestationis'
 'acrodermatitis enteropathica' 'actinic keratosis'
 'allergic contact dermatitis' 'aplasia cutis' 'basal cell carcinoma'
 'basal cell carcinoma morpheiform' 'becker nevus' 'behcets disease'
 'calcinosis cutis' 'cheilitis' 'congenital nevus' 'dariers disease'
 'dermatofibroma' 'dermatomyositis' 'disseminated actinic porokeratosis'
 'drug eruption' 'drug induced pigmentary changes' 'dyshidrotic eczema'
 'eczema' 'ehlers danlos syndrome' 'epidermal nevus'
 'epidermolysis bullosa' 'erythema annulare centrifigum'
 'erythema elevatum diutinum' 'erythema multiforme' 'erythema nodosum'
 'factitial dermatitis' 'fixed eruptions' 'folliculitis' 'fordyce spots'
 'granuloma annulare' 'granuloma pyogenic' 'hailey hailey disease'
 'halo nevus' 'hidradenitis' 'ichthyosis vulgaris'
 'incontinentia pigmenti' 'juvenile xanthogranuloma' 'kaposi sarcoma'
 'keloid' 'keratosis pilaris' 'lan

In [52]:
import cv2

# List to store the paths of .npz files
npz_paths = []

# Loop over the rows of the DataFrame
for i, row in df.iterrows():
    # Get the image path and .npz file path
    picture_path = row['image_path']
    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    # Load the image
    pic_bgr_arr = cv2.imread(picture_path)
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    # Extract labels for the nine partitions
    ben_derm, ben_epiderm, ben_mel = row['nine_partition_label_benign dermal'], row['nine_partition_label_benign epidermal'], row['nine_partition_label_benign melanocyte']
    gen, inf = row['nine_partition_label_genodermatoses'], row['nine_partition_label_inflammatory']
    mal_cut, mal_der, mal_epider, mal_mel = row['nine_partition_label_malignant cutaneous lymphoma'], row['nine_partition_label_malignant dermal'], row['nine_partition_label_malignant epidermal'], row['nine_partition_label_malignant melanoma']

    # Create an array with the labels for the nine partitions
    nines = np.array([ben_derm, ben_epiderm, ben_mel, gen, inf, mal_cut, mal_der, mal_epider, mal_mel])
        
    # Get the overall label
    label = row['label']

    # Save the data into the .npz file
    np.savez_compressed(npz_path, pic=pic_rgb_arr, nines=nines, label=label)

# Add the NPZ file paths column to the DataFrame
df['NPZ_Path'] = pd.Series(npz_paths)


In [53]:
# Create a nine_partition_feature df
nines_df=df[['nine_partition_label_benign dermal', 'nine_partition_label_benign epidermal', 'nine_partition_label_benign melanocyte', 'nine_partition_label_genodermatoses', 'nine_partition_label_inflammatory', 'nine_partition_label_malignant cutaneous lymphoma', 'nine_partition_label_malignant dermal', 'nine_partition_label_malignant epidermal', 'nine_partition_label_malignant melanoma']]

In [54]:
# Drop the nine_partition columns
df = df.drop(columns=df.columns.difference(['image_name', 'label', 'NPZ_Path']))
df

,image_name,label,NPZ_Path
0,drug induced pigmentary changes_0,20,images_resized_full\drug induced pigmentary ch...
1,photodermatoses_0,75,images_resized_full\photodermatoses_0.npz
2,dermatofibroma_0,16,images_resized_full\dermatofibroma_0.npz
3,psoriasis_0,86,images_resized_full\psoriasis_0.npz
4,psoriasis_1,86,images_resized_full\psoriasis_1.npz
...,...,...,...
16365,scleroderma_306,93,images_resized_full\scleroderma_306.npz
16366,pityriasis rosea_190,79,images_resized_full\pityriasis rosea_190.npz
16367,myiasis_66,61,images_resized_full\myiasis_66.npz
16368,vitiligo_164,111,images_resized_full\vitiligo_164.npz


In [55]:
shuffled_df = df.sample(frac=1)

# Calculate the size of each set based on desired proportions
total_samples = len(shuffled_df)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

# Divide the DataFrame into training, validation, and test sets
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:train_size+val_size]
test_df = shuffled_df[train_size+val_size:]

len(train_df), len(val_df), len(test_df)

(11459, 2455, 2456)

In [56]:
def load_npz_data(npz_file):
    """Load data from a .npz file."""
    with np.load(npz_file, allow_pickle=True) as data:
        pic = data['pic']
        nines = data['nines']
        label = data['label']
    return pic, nines, label



def get_X_y(df):
    X_pic, X_nines = [], []
    y = []

    for name in df['NPZ_Path']:
        loaded_npz = np.load(name)

        pic = loaded_npz['pic']
        X_pic.append(pic)

        nines = loaded_npz['nines']
        X_nines.append(nines)

        y.append(loaded_npz['label'])

    X_pic, X_nines = np.array(X_pic), np.array(X_nines)
    y = np.array(y)

    return (X_pic, X_nines), y

(X_train_pic, X_train_nines), y_train = get_X_y(train_df)
(X_train_pic.shape, X_train_nines.shape), y_train.shape


(((11459, 96, 96, 3), (11459, 9)), (11459,))

In [57]:
(X_val_pic, X_val_nines), y_val = get_X_y(val_df)
(X_val_pic.shape, X_val_nines.shape), y_val.shape

(((2455, 96, 96, 3), (2455, 9)), (2455,))

In [58]:
(X_test_pic, X_test_nines), y_test = get_X_y(test_df)
(X_test_pic.shape, X_test_nines.shape), y_test.shape

(((2456, 96, 96, 3), (2456, 9)), (2456,))

In [59]:
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

def build_model(conv_params, dense_params, learning_rate=0.0001, dropout=0):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
    pic_output = x

    
    # Stats stream 
    input_stats = layers.Input(shape=(9,))  
    y = input_stats
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output = y  

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model
model = build_model(conv_params, dense_params)


In [60]:
from tensorflow.keras.layers import Dropout

def build_model_dropout(conv_params, dense_params, learning_rate=0.0001, dropout=0.5):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each convolutional layer
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each dense layer
    pic_output = x

    
    # Stats stream 
    input_stats = layers.Input(shape=(9,))  
    y = input_stats
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output = y  

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model with dropout rate of 0.5
model_dropout = build_model_dropout(conv_params, dense_params, dropout=0.5)

In [61]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_3 (Conv2D)           (None, 94, 94, 32)           896       ['input_3[0][0]']             
                                                                                                  
 max_pooling2d_3 (MaxPoolin  (None, 47, 47, 32)           0         ['conv2d_3[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_4 (Conv2D)           (None, 45, 45, 64)           18496     ['max_pooling2d_3[0][0]'

In [62]:
model_dropout.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 94, 94, 32)           896       ['input_5[0][0]']             
                                                                                                  
 max_pooling2d_6 (MaxPoolin  (None, 47, 47, 32)           0         ['conv2d_6[0][0]']            
 g2D)                                                                                             
                                                                                                  
 dropout (Dropout)           (None, 47, 47, 32)           0         ['max_pooling2d_6[0][0]'

In [63]:
from keras.callbacks import ModelCheckpoint

cp = ModelCheckpoint('model_nines/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_nines], y=y_train,
          validation_data=([X_val_pic, X_val_nines], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 5.0031 - accuracy: 0.0516INFO:tensorflow:Assets written to: model_nines\assets


INFO:tensorflow:Assets written to: model_nines\assets


359/359 [==============================] - 103s 282ms/step - loss: 5.0031 - accuracy: 0.0516 - val_loss: 4.5883 - val_accuracy: 0.0782
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.4713 - accuracy: 0.1123INFO:tensorflow:Assets written to: model_nines\assets


INFO:tensorflow:Assets written to: model_nines\assets


359/359 [==============================] - 112s 313ms/step - loss: 4.4713 - accuracy: 0.1123 - val_loss: 4.3385 - val_accuracy: 0.1544
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.1855 - accuracy: 0.1422INFO:tensorflow:Assets written to: model_nines\assets


...

359/359 [==============================] - 123s 343ms/step - loss: 3.2333 - accuracy: 0.1447 - val_loss: 3.2155 - val_accuracy: 0.1556
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 3.2317 - accuracy: 0.1450INFO:tensorflow:Assets written to: model_nines\assets


INFO:tensorflow:Assets written to: model_nines\assets


359/359 [==============================] - 116s 324ms/step - loss: 3.2317 - accuracy: 0.1450 - val_loss: 3.2138 - val_accuracy: 0.1536
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 3.2305 - accuracy: 0.1439INFO:tensorflow:Assets written to: model_nines\assets


INFO:tensorflow:Assets written to: model_nines\assets


359/359 [==============================] - 117s 327ms/step - loss: 3.2305 - accuracy: 0.1439 - val_loss: 3.2129 - val_accuracy: 0.1556


In [64]:
cp = ModelCheckpoint('model_nines_dropout/', save_best_only=True)

model_dropout.fit(x=[X_train_pic, X_train_nines], y=y_train,
          validation_data=([X_val_pic, X_val_nines], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 8.4007 - accuracy: 0.0401INFO:tensorflow:Assets written to: model_nines_dropout\assets


INFO:tensorflow:Assets written to: model_nines_dropout\assets


359/359 [==============================] - 134s 368ms/step - loss: 8.4007 - accuracy: 0.0401 - val_loss: 4.6127 - val_accuracy: 0.0925
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.5338 - accuracy: 0.1012INFO:tensorflow:Assets written to: model_nines_dropout\assets


INFO:tensorflow:Assets written to: model_nines_dropout\assets


359/359 [==============================] - 134s 374ms/step - loss: 4.5338 - accuracy: 0.1012 - val_loss: 4.4209 - val_accuracy: 0.1365
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.3084 - accuracy: 0.1225INFO:tensorflow:Assets written to: model_nines_dropout\assets


...

359/359 [==============================] - 129s 359ms/step - loss: 3.2655 - accuracy: 0.1412 - val_loss: 3.2192 - val_accuracy: 0.1556
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 3.2665 - accuracy: 0.1346INFO:tensorflow:Assets written to: model_nines_dropout\assets


INFO:tensorflow:Assets written to: model_nines_dropout\assets


359/359 [==============================] - 142s 395ms/step - loss: 3.2665 - accuracy: 0.1346 - val_loss: 3.2170 - val_accuracy: 0.1556
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 3.2609 - accuracy: 0.1402INFO:tensorflow:Assets written to: model_nines_dropout\assets


INFO:tensorflow:Assets written to: model_nines_dropout\assets


359/359 [==============================] - 139s 386ms/step - loss: 3.2609 - accuracy: 0.1402 - val_loss: 3.2155 - val_accuracy: 0.1556


In [65]:
from tensorflow.keras.models import load_model

loaded_model = load_model('model_nines/')

In [66]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_nines])
test_predictions

77/77 [==============================] - 8s 95ms/step


array([[3.7958707e-05, 2.8866258e-05, 4.4046814e-05, ..., 3.3627050e-05,
        2.3159037e-05, 3.1018066e-05],
       [8.7644802e-03, 1.9094381e-02, 2.9992113e-02, ..., 1.4625360e-02,
        4.5340303e-03, 3.8920989e-05],
       [8.7644802e-03, 1.9094381e-02, 2.9992113e-02, ..., 1.4625360e-02,
        4.5340303e-03, 3.8920989e-05],
       ...,
       [8.7644802e-03, 1.9094381e-02, 2.9992113e-02, ..., 1.4625360e-02,
        4.5340303e-03, 3.8920989e-05],
       [1.6941896e-04, 1.6158386e-04, 2.5183716e-04, ..., 1.4132478e-04,
        1.1441027e-04, 1.4311980e-04],
       [8.7644802e-03, 1.9094381e-02, 2.9992113e-02, ..., 1.4625360e-02,
        4.5340303e-03, 3.8920989e-05]], dtype=float32)

In [67]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([98, 86, 86, ..., 86, 81, 86], dtype=int64)

In [68]:
true_labels = test_df['label']

In [69]:
from sklearn.metrics import f1_score

f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.05613799818661296

In [70]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.13436482084690554

In [71]:
# Evaluate the model on the test set
test_loss = model.evaluate(x=[X_test_pic, X_test_nines], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 7s 95ms/step - loss: 3.2305 - accuracy: 0.1344
Test Loss: [3.2304956912994385, 0.13436481356620789]


In [72]:
from tensorflow.keras.models import load_model

loaded_model = load_model('model_nines_dropout/')

In [73]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_nines])
test_predictions

77/77 [==============================] - 8s 96ms/step


array([[4.73134569e-05, 6.42377709e-05, 6.55253025e-05, ...,
        5.57849853e-05, 2.49702025e-05, 5.11021171e-05],
       [8.98874924e-03, 1.89436842e-02, 2.97243893e-02, ...,
        1.48363365e-02, 4.52825567e-03, 6.85944906e-05],
       [8.98874924e-03, 1.89436842e-02, 2.97243893e-02, ...,
        1.48363365e-02, 4.52825567e-03, 6.85944906e-05],
       ...,
       [8.98874924e-03, 1.89436842e-02, 2.97243893e-02, ...,
        1.48363365e-02, 4.52825567e-03, 6.85944906e-05],
       [8.80391890e-05, 7.16829454e-05, 1.32867775e-04, ...,
        9.34692289e-05, 6.57618657e-05, 1.03160797e-04],
       [8.98874924e-03, 1.89436842e-02, 2.97243893e-02, ...,
        1.48363365e-02, 4.52825567e-03, 6.85944906e-05]], dtype=float32)

In [74]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([98, 86, 86, ..., 86, 81, 86], dtype=int64)

In [75]:
true_labels = test_df['label']

In [76]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.05613799818661296

In [77]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.13436482084690554

In [78]:
# Evaluate the model on the test set
test_loss = model_dropout.evaluate(x=[X_test_pic, X_test_nines], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 7s 95ms/step - loss: 3.2327 - accuracy: 0.1344
Test Loss: [3.2326598167419434, 0.13436481356620789]


**CNN + three_partition_label**

In [79]:
df = pd.read_csv('fitzpatrick17k_3')
df

,image_name,image_path,label,fitzpatrick_scale,fitzpatrick_centaur,qc,url,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,drug induced pigmentary changes_0,images_full\drug induced pigmentary changes_0.jpg,drug induced pigmentary changes,3,3,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
1,photodermatoses_0,images_full\photodermatoses_0.jpg,photodermatoses,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
2,dermatofibroma_0,images_full\dermatofibroma_0.jpg,dermatofibroma,2,3,NaN,https://www.dermaamin.com/site/images/clinical...,1,0,0,0,0,0,0,0,0,1,0,0
3,psoriasis_0,images_full\psoriasis_0.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
4,psoriasis_1,images_full\psoriasis_1.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,scleroderma_306,images_full\scleroderma_306.jpg,scleroderma,5,5,NaN,http://atlasdermatologico.com.br/img?imageId=4529,0,0,0,0,1,0,0,0,0,0,0,1
16366,pityriasis rosea_190,images_full\pityriasis rosea_190.jpg,pityriasis rosea,4,5,NaN,http://atlasdermatologico.com.br/img?imageId=5630,0,0,0,0,1,0,0,0,0,0,0,1
16367,myiasis_66,images_full\myiasis_66.jpg,myiasis,3,3,NaN,http://atlasdermatologico.com.br/img?imageId=4609,0,0,0,0,1,0,0,0,0,0,0,1
16368,vitiligo_164,images_full\vitiligo_164.jpg,vitiligo,-1,5,NaN,http://atlasdermatologico.com.br/img?imageId=7930,0,0,0,0,1,0,0,0,0,0,0,1


In [80]:
df = df.drop(['fitzpatrick_centaur', 'fitzpatrick_scale','qc', 'url', 'nine_partition_label_benign dermal', 'nine_partition_label_benign epidermal', 'nine_partition_label_benign melanocyte', 'nine_partition_label_genodermatoses', 'nine_partition_label_inflammatory','nine_partition_label_malignant cutaneous lymphoma', 'nine_partition_label_malignant dermal', 'nine_partition_label_malignant epidermal', 'nine_partition_label_malignant melanoma'], axis=1)

In [81]:
# Replace 'images' with 'images_resized' in the 'image_path' column
df['image_path'] = df['image_path'].str.replace('images_full', 'images_resized_full')

In [82]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Visualize the unique and encoded labels
print("Unique labels:", label_encoder.classes_)
print("Encoded labels:", df['label'].unique())

Unique labels: ['acanthosis nigricans' 'acne' 'acne vulgaris'
 'acquired autoimmune bullous diseaseherpes gestationis'
 'acrodermatitis enteropathica' 'actinic keratosis'
 'allergic contact dermatitis' 'aplasia cutis' 'basal cell carcinoma'
 'basal cell carcinoma morpheiform' 'becker nevus' 'behcets disease'
 'calcinosis cutis' 'cheilitis' 'congenital nevus' 'dariers disease'
 'dermatofibroma' 'dermatomyositis' 'disseminated actinic porokeratosis'
 'drug eruption' 'drug induced pigmentary changes' 'dyshidrotic eczema'
 'eczema' 'ehlers danlos syndrome' 'epidermal nevus'
 'epidermolysis bullosa' 'erythema annulare centrifigum'
 'erythema elevatum diutinum' 'erythema multiforme' 'erythema nodosum'
 'factitial dermatitis' 'fixed eruptions' 'folliculitis' 'fordyce spots'
 'granuloma annulare' 'granuloma pyogenic' 'hailey hailey disease'
 'halo nevus' 'hidradenitis' 'ichthyosis vulgaris'
 'incontinentia pigmenti' 'juvenile xanthogranuloma' 'kaposi sarcoma'
 'keloid' 'keratosis pilaris' 'lan

In [83]:
df

,image_name,image_path,label,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,drug induced pigmentary changes_0,images_resized_full\drug induced pigmentary ch...,20,0,0,1
1,photodermatoses_0,images_resized_full\photodermatoses_0.jpg,75,0,0,1
2,dermatofibroma_0,images_resized_full\dermatofibroma_0.jpg,16,1,0,0
3,psoriasis_0,images_resized_full\psoriasis_0.jpg,86,0,0,1
4,psoriasis_1,images_resized_full\psoriasis_1.jpg,86,0,0,1
...,...,...,...,...,...,...
16365,scleroderma_306,images_resized_full\scleroderma_306.jpg,93,0,0,1
16366,pityriasis rosea_190,images_resized_full\pityriasis rosea_190.jpg,79,0,0,1
16367,myiasis_66,images_resized_full\myiasis_66.jpg,61,0,0,1
16368,vitiligo_164,images_resized_full\vitiligo_164.jpg,111,0,0,1


In [84]:
import numpy as np
import cv2

# List to store paths of .npz files
npz_paths = []

# Loop over the DataFrame rows
for i, row in df.iterrows():
    # Get the image path and .npz file path
    picture_path = row['image_path']
    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    # Load the image
    pic_bgr_arr = cv2.imread(picture_path)
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    # Extracting the three_partition labels
    ben, mal, non_neo = row['three_partition_label_benign'], row['three_partition_label_malignant'], row['three_partition_label_non-neoplastic']
    
    # Creating an array for three_partition labels
    threes = np.array([ben, mal, non_neo])
    
    # Getting the overall label
    label = row['label']

    # Save the data into the .npz file
    np.savez_compressed(npz_path, pic=pic_rgb_arr, threes=threes, label=label)

# Add the NPZ paths column to the DataFrame
df['NPZ_Path'] = pd.Series(npz_paths)

In [85]:
threes_df = df[['three_partition_label_benign', 'three_partition_label_malignant', 'three_partition_label_non-neoplastic']]

In [86]:
df = df.drop(columns=df.columns.difference(['image_name', 'label', 'NPZ_Path']))
df

,image_name,label,NPZ_Path
0,drug induced pigmentary changes_0,20,images_resized_full\drug induced pigmentary ch...
1,photodermatoses_0,75,images_resized_full\photodermatoses_0.npz
2,dermatofibroma_0,16,images_resized_full\dermatofibroma_0.npz
3,psoriasis_0,86,images_resized_full\psoriasis_0.npz
4,psoriasis_1,86,images_resized_full\psoriasis_1.npz
...,...,...,...
16365,scleroderma_306,93,images_resized_full\scleroderma_306.npz
16366,pityriasis rosea_190,79,images_resized_full\pityriasis rosea_190.npz
16367,myiasis_66,61,images_resized_full\myiasis_66.npz
16368,vitiligo_164,111,images_resized_full\vitiligo_164.npz


In [87]:
shuffled_df = df.sample(frac=1)

# Calculate the size of each set based on desired proportions
total_samples = len(shuffled_df)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

# Divide the DataFrame into training, validation, and test sets
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:train_size+val_size]
test_df = shuffled_df[train_size+val_size:]

len(train_df), len(val_df), len(test_df)

(11459, 2455, 2456)

In [88]:
def load_npz_data(npz_file):
    """Load data from a .npz file."""
    with np.load(npz_file, allow_pickle=True) as data:
        pic = data['pic']
        threes = data['threes']
        label = data['label']
    return pic, threes, label



def get_X_y(df):
    X_pic, X_threes = [], []
    y = []

    for name in df['NPZ_Path']:
        loaded_npz = np.load(name)

        pic = loaded_npz['pic']
        X_pic.append(pic)

        threes = loaded_npz['threes']
        X_threes.append(threes)

        y.append(loaded_npz['label'])

    X_pic, X_threes = np.array(X_pic), np.array(X_threes)
    y = np.array(y)

    return (X_pic, X_threes), y

(X_train_pic, X_train_threes), y_train = get_X_y(train_df)
(X_train_pic.shape, X_train_threes.shape), y_train.shape

(((11459, 96, 96, 3), (11459, 3)), (11459,))

In [89]:
(X_val_pic, X_val_threes), y_val = get_X_y(val_df)
(X_val_pic.shape, X_val_threes.shape), y_val.shape

(((2455, 96, 96, 3), (2455, 3)), (2455,))

In [90]:
(X_test_pic, X_test_threes), y_test = get_X_y(test_df)
(X_test_pic.shape, X_test_threes.shape), y_test.shape

(((2456, 96, 96, 3), (2456, 3)), (2456,))

In [91]:
def build_model(conv_params, dense_params, learning_rate=0.0001, dropout=0):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
    pic_output = x

    
    # Stats stream 
    input_stats = layers.Input(shape=(3,))  
    y = input_stats
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output = y  

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model
model = build_model(conv_params, dense_params)

In [92]:
from tensorflow.keras.layers import Dropout

def build_model_dropout(conv_params, dense_params, learning_rate=0.0001, dropout=0.5):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each convolutional layer
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each dense layer
    pic_output = x

    
    # Stats stream 
    input_stats = layers.Input(shape=(3,))  
    y = input_stats
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output = y  

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model with dropout rate of 0.5
model_dropout = build_model_dropout(conv_params, dense_params, dropout=0.5)

In [93]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_9 (Conv2D)           (None, 94, 94, 32)           896       ['input_7[0][0]']             
                                                                                                  
 max_pooling2d_9 (MaxPoolin  (None, 47, 47, 32)           0         ['conv2d_9[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_10 (Conv2D)          (None, 45, 45, 64)           18496     ['max_pooling2d_9[0][0]'

In [94]:
model_dropout.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_12 (Conv2D)          (None, 94, 94, 32)           896       ['input_9[0][0]']             
                                                                                                  
 max_pooling2d_12 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_12[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 dropout_5 (Dropout)         (None, 47, 47, 32)           0         ['max_pooling2d_12[0][0]

In [95]:
cp = ModelCheckpoint('model_threes/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_threes], y=y_train,
          validation_data=([X_val_pic, X_val_threes], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 5.2170 - accuracy: 0.0533INFO:tensorflow:Assets written to: model_threes\assets


INFO:tensorflow:Assets written to: model_threes\assets


359/359 [==============================] - 116s 320ms/step - loss: 5.2170 - accuracy: 0.0533 - val_loss: 4.5781 - val_accuracy: 0.0798
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.4787 - accuracy: 0.0792INFO:tensorflow:Assets written to: model_threes\assets


INFO:tensorflow:Assets written to: model_threes\assets


359/359 [==============================] - 118s 329ms/step - loss: 4.4787 - accuracy: 0.0792 - val_loss: 4.3516 - val_accuracy: 0.0798
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.2360 - accuracy: 0.0796INFO:tensorflow:Assets written to: model_threes\assets


...

359/359 [==============================] - 115s 322ms/step - loss: 3.7383 - accuracy: 0.0880 - val_loss: 3.7200 - val_accuracy: 0.0827
Epoch 18/20
359/359 [==============================] - ETA: 0s - loss: 3.7350 - accuracy: 0.0884INFO:tensorflow:Assets written to: model_threes\assets


INFO:tensorflow:Assets written to: model_threes\assets


359/359 [==============================] - 115s 321ms/step - loss: 3.7350 - accuracy: 0.0884 - val_loss: 3.7184 - val_accuracy: 0.0827
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 3.7333 - accuracy: 0.0885INFO:tensorflow:Assets written to: model_threes\assets


INFO:tensorflow:Assets written to: model_threes\assets


359/359 [==============================] - 115s 322ms/step - loss: 3.7333 - accuracy: 0.0885 - val_loss: 3.7171 - val_accuracy: 0.0827
Epoch 20/20
359/359 [==============================] - 114s 318ms/step - loss: 3.7326 - accuracy: 0.0882 - val_loss: 3.7224 - val_accuracy: 0.0831


In [96]:
cp = ModelCheckpoint('model_threes_dropout/', save_best_only=True)

model_dropout.fit(x=[X_train_pic, X_train_threes], y=y_train,
          validation_data=([X_val_pic, X_val_threes], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 8.5950 - accuracy: 0.0271INFO:tensorflow:Assets written to: model_threes_dropout\assets


INFO:tensorflow:Assets written to: model_threes_dropout\assets


359/359 [==============================] - 130s 358ms/step - loss: 8.5950 - accuracy: 0.0271 - val_loss: 4.6384 - val_accuracy: 0.0741
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.5817 - accuracy: 0.0596INFO:tensorflow:Assets written to: model_threes_dropout\assets


INFO:tensorflow:Assets written to: model_threes_dropout\assets


359/359 [==============================] - 128s 356ms/step - loss: 4.5817 - accuracy: 0.0596 - val_loss: 4.4755 - val_accuracy: 0.0741
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.3828 - accuracy: 0.0688INFO:tensorflow:Assets written to: model_threes_dropout\assets


...

359/359 [==============================] - 129s 359ms/step - loss: 3.7561 - accuracy: 0.0819 - val_loss: 3.7189 - val_accuracy: 0.0827
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 3.7580 - accuracy: 0.0853INFO:tensorflow:Assets written to: model_threes_dropout\assets


INFO:tensorflow:Assets written to: model_threes_dropout\assets


359/359 [==============================] - 143s 400ms/step - loss: 3.7580 - accuracy: 0.0853 - val_loss: 3.7176 - val_accuracy: 0.0827
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 3.7613 - accuracy: 0.0835INFO:tensorflow:Assets written to: model_threes_dropout\assets


INFO:tensorflow:Assets written to: model_threes_dropout\assets


359/359 [==============================] - 152s 424ms/step - loss: 3.7613 - accuracy: 0.0835 - val_loss: 3.7167 - val_accuracy: 0.0827


In [97]:
loaded_model = load_model('model_threes/')

In [98]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_threes])
test_predictions

77/77 [==============================] - 8s 104ms/step


array([[7.0591192e-03, 1.4592950e-02, 2.8802136e-02, ..., 1.4243548e-02,
        4.7363555e-03, 6.4673233e-03],
       [7.0591192e-03, 1.4592950e-02, 2.8802136e-02, ..., 1.4243548e-02,
        4.7363555e-03, 6.4673233e-03],
       [7.0591192e-03, 1.4592950e-02, 2.8802136e-02, ..., 1.4243548e-02,
        4.7363555e-03, 6.4673233e-03],
       ...,
       [7.0591192e-03, 1.4592950e-02, 2.8802136e-02, ..., 1.4243548e-02,
        4.7363555e-03, 6.4673233e-03],
       [3.8157137e-05, 3.4811717e-05, 6.5633452e-05, ..., 4.0600979e-05,
        3.2686869e-05, 3.5775269e-05],
       [7.0591192e-03, 1.4592950e-02, 2.8802136e-02, ..., 1.4243548e-02,
        4.7363555e-03, 6.4673233e-03]], dtype=float32)

In [99]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([86, 86, 86, ..., 86, 98, 86], dtype=int64)

In [100]:
true_labels = test_df['label']

In [101]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.017107244054036682

In [102]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.07776872964169382

In [103]:
# Evaluate the model on the test set
test_loss = model.evaluate(x=[X_test_pic, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 8s 106ms/step - loss: 3.7535 - accuracy: 0.0782
Test Loss: [3.7534704208374023, 0.07817589491605759]


In [104]:
loaded_model = load_model('model_threes_dropout/')

In [105]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_threes])
test_predictions

77/77 [==============================] - 8s 103ms/step


array([[7.22016161e-03, 1.45118255e-02, 2.82861702e-02, ...,
        1.40139461e-02, 4.70740953e-03, 6.53596316e-03],
       [7.22016161e-03, 1.45118255e-02, 2.82861702e-02, ...,
        1.40139461e-02, 4.70740953e-03, 6.53596316e-03],
       [7.22016161e-03, 1.45118255e-02, 2.82861702e-02, ...,
        1.40139461e-02, 4.70740953e-03, 6.53596316e-03],
       ...,
       [7.22016161e-03, 1.45118255e-02, 2.82861702e-02, ...,
        1.40139461e-02, 4.70740953e-03, 6.53596316e-03],
       [4.80924136e-05, 7.07027284e-05, 7.72307176e-05, ...,
        8.63433088e-05, 3.56341516e-05, 5.45281764e-05],
       [7.22016161e-03, 1.45118255e-02, 2.82861702e-02, ...,
        1.40139461e-02, 4.70740953e-03, 6.53596316e-03]], dtype=float32)

In [106]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([86, 86, 86, ..., 86, 98, 86], dtype=int64)

In [107]:
true_labels = test_df['label']

In [108]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.017105224369919157

In [109]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.07776872964169382

In [110]:
# Evaluate the model on the test set
test_loss = model_dropout.evaluate(x=[X_test_pic, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 8s 105ms/step - loss: 3.7508 - accuracy: 0.0778
Test Loss: [3.7508156299591064, 0.0777687281370163]


**CNN + fitzpatrick_scale**

In [111]:
df = pd.read_csv('fitzpatrick17k_3')
df

,image_name,image_path,label,fitzpatrick_scale,fitzpatrick_centaur,qc,url,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,drug induced pigmentary changes_0,images_full\drug induced pigmentary changes_0.jpg,drug induced pigmentary changes,3,3,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
1,photodermatoses_0,images_full\photodermatoses_0.jpg,photodermatoses,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
2,dermatofibroma_0,images_full\dermatofibroma_0.jpg,dermatofibroma,2,3,NaN,https://www.dermaamin.com/site/images/clinical...,1,0,0,0,0,0,0,0,0,1,0,0
3,psoriasis_0,images_full\psoriasis_0.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
4,psoriasis_1,images_full\psoriasis_1.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,scleroderma_306,images_full\scleroderma_306.jpg,scleroderma,5,5,NaN,http://atlasdermatologico.com.br/img?imageId=4529,0,0,0,0,1,0,0,0,0,0,0,1
16366,pityriasis rosea_190,images_full\pityriasis rosea_190.jpg,pityriasis rosea,4,5,NaN,http://atlasdermatologico.com.br/img?imageId=5630,0,0,0,0,1,0,0,0,0,0,0,1
16367,myiasis_66,images_full\myiasis_66.jpg,myiasis,3,3,NaN,http://atlasdermatologico.com.br/img?imageId=4609,0,0,0,0,1,0,0,0,0,0,0,1
16368,vitiligo_164,images_full\vitiligo_164.jpg,vitiligo,-1,5,NaN,http://atlasdermatologico.com.br/img?imageId=7930,0,0,0,0,1,0,0,0,0,0,0,1


In [112]:
df = df.drop(['fitzpatrick_centaur', 'qc', 'url', 'nine_partition_label_benign dermal', 'nine_partition_label_benign epidermal', 'nine_partition_label_benign melanocyte', 'nine_partition_label_genodermatoses', 'nine_partition_label_inflammatory','nine_partition_label_malignant cutaneous lymphoma', 'nine_partition_label_malignant dermal', 'nine_partition_label_malignant epidermal', 'nine_partition_label_malignant melanoma', 'three_partition_label_benign', 'three_partition_label_malignant', 'three_partition_label_non-neoplastic'], axis=1)

In [113]:
# Replace 'images' with 'images_resized' in the 'image_path' column
df['image_path'] = df['image_path'].str.replace('images_full', 'images_resized_full')

In [114]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Visualize the unique and encoded labels
print("Unique labels:", label_encoder.classes_)
print("Encoded labels:", df['label'].unique())

Unique labels: ['acanthosis nigricans' 'acne' 'acne vulgaris'
 'acquired autoimmune bullous diseaseherpes gestationis'
 'acrodermatitis enteropathica' 'actinic keratosis'
 'allergic contact dermatitis' 'aplasia cutis' 'basal cell carcinoma'
 'basal cell carcinoma morpheiform' 'becker nevus' 'behcets disease'
 'calcinosis cutis' 'cheilitis' 'congenital nevus' 'dariers disease'
 'dermatofibroma' 'dermatomyositis' 'disseminated actinic porokeratosis'
 'drug eruption' 'drug induced pigmentary changes' 'dyshidrotic eczema'
 'eczema' 'ehlers danlos syndrome' 'epidermal nevus'
 'epidermolysis bullosa' 'erythema annulare centrifigum'
 'erythema elevatum diutinum' 'erythema multiforme' 'erythema nodosum'
 'factitial dermatitis' 'fixed eruptions' 'folliculitis' 'fordyce spots'
 'granuloma annulare' 'granuloma pyogenic' 'hailey hailey disease'
 'halo nevus' 'hidradenitis' 'ichthyosis vulgaris'
 'incontinentia pigmenti' 'juvenile xanthogranuloma' 'kaposi sarcoma'
 'keloid' 'keratosis pilaris' 'lan

In [115]:
df = pd.get_dummies(df, columns=['fitzpatrick_scale'])

In [116]:
import cv2

npz_paths = []

# Loop over the DataFrame rows
for i, row in df.iterrows():
    # Get the image path and .npz file path
    picture_path = row['image_path']
    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    # Load the image
    pic_bgr_arr = cv2.imread(picture_path)
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    # Extracting Fitzpatrick scale values
    minus_one_scale, ones_cale, two_scale = row['fitzpatrick_scale_-1'], row['fitzpatrick_scale_1'], row['fitzpatrick_scale_2']
    three_scale, four_scale, five_scale, six_scale = row['fitzpatrick_scale_3'], row['fitzpatrick_scale_4'], row['fitzpatrick_scale_5'], row['fitzpatrick_scale_6']
   
    # Creating an array for Fitzpatrick scale values
    scale = np.array([minus_one_scale, ones_cale,  two_scale, three_scale, four_scale, five_scale, six_scale])
    
    # Getting the overall label
    label = row['label']

    # Save the data into the .npz file
    np.savez_compressed(npz_path, pic=pic_rgb_arr, scale=scale, label=label)

# Add the NPZ paths column to the DataFrame
df['NPZ_Path'] = pd.Series(npz_paths)

In [117]:
scale_df = df.iloc[:, 3:]
scale_df

,fitzpatrick_scale_-1,fitzpatrick_scale_1,fitzpatrick_scale_2,fitzpatrick_scale_3,fitzpatrick_scale_4,fitzpatrick_scale_5,fitzpatrick_scale_6,NPZ_Path
0,0,0,0,1,0,0,0,images_resized_full\drug induced pigmentary ch...
1,0,1,0,0,0,0,0,images_resized_full\photodermatoses_0.npz
2,0,0,1,0,0,0,0,images_resized_full\dermatofibroma_0.npz
3,0,1,0,0,0,0,0,images_resized_full\psoriasis_0.npz
4,0,1,0,0,0,0,0,images_resized_full\psoriasis_1.npz
...,...,...,...,...,...,...,...,...
16365,0,0,0,0,0,1,0,images_resized_full\scleroderma_306.npz
16366,0,0,0,0,1,0,0,images_resized_full\pityriasis rosea_190.npz
16367,0,0,0,1,0,0,0,images_resized_full\myiasis_66.npz
16368,1,0,0,0,0,0,0,images_resized_full\vitiligo_164.npz


In [118]:
df = df.drop(columns=df.columns.difference(['image_name', 'label', 'NPZ_Path']))
df

,image_name,label,NPZ_Path
0,drug induced pigmentary changes_0,20,images_resized_full\drug induced pigmentary ch...
1,photodermatoses_0,75,images_resized_full\photodermatoses_0.npz
2,dermatofibroma_0,16,images_resized_full\dermatofibroma_0.npz
3,psoriasis_0,86,images_resized_full\psoriasis_0.npz
4,psoriasis_1,86,images_resized_full\psoriasis_1.npz
...,...,...,...
16365,scleroderma_306,93,images_resized_full\scleroderma_306.npz
16366,pityriasis rosea_190,79,images_resized_full\pityriasis rosea_190.npz
16367,myiasis_66,61,images_resized_full\myiasis_66.npz
16368,vitiligo_164,111,images_resized_full\vitiligo_164.npz


In [119]:
shuffled_df = df.sample(frac=1)

# Calculate the size of each set based on desired proportions
total_samples = len(shuffled_df)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

# Divide the DataFrame into training, validation, and test sets
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:train_size+val_size]
test_df = shuffled_df[train_size+val_size:]

len(train_df), len(val_df), len(test_df)

(11459, 2455, 2456)

In [120]:
def load_npz_data(npz_file):
    """Load data from a .npz file."""
    with np.load(npz_file, allow_pickle=True) as data:
        pic = data['pic']
        scale = data['scale']
        label = data['label']
    return pic, scale, label



def get_X_y(df):
    X_pic, X_scale = [], []
    y = []

    for name in df['NPZ_Path']:
        loaded_npz = np.load(name)

        pic = loaded_npz['pic']
        X_pic.append(pic)

        scale = loaded_npz['scale']
        X_scale.append(scale)

        y.append(loaded_npz['label'])

    X_pic, X_scale = np.array(X_pic), np.array(X_scale)
    y = np.array(y)

    return (X_pic, X_scale), y

(X_train_pic, X_train_scale), y_train = get_X_y(train_df)
(X_train_pic.shape, X_train_scale.shape), y_train.shape

(((11459, 96, 96, 3), (11459, 7)), (11459,))

In [121]:
(X_val_pic, X_val_scale), y_val = get_X_y(val_df)
(X_val_pic.shape, X_val_scale.shape), y_val.shape

(((2455, 96, 96, 3), (2455, 7)), (2455,))

In [122]:
(X_test_pic, X_test_scale), y_test = get_X_y(test_df)
(X_test_pic.shape, X_test_scale.shape), y_test.shape

(((2456, 96, 96, 3), (2456, 7)), (2456,))

In [123]:
def build_model(conv_params, dense_params, learning_rate=0.0001, dropout=0):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
    pic_output = x

    
    # Stats stream 
    input_stats = layers.Input(shape=(7,))  
    y = input_stats
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output = y  

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model
model = build_model(conv_params, dense_params)

In [124]:
from tensorflow.keras.layers import Dropout

def build_model_dropout(conv_params, dense_params, learning_rate=0.0001, dropout=0.5):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each convolutional layer
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each dense layer
    pic_output = x

    
    # Stats stream 
    input_stats = layers.Input(shape=(7,))  
    y = input_stats
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output = y  

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model with dropout rate of 0.5
model_dropout = build_model_dropout(conv_params, dense_params, dropout=0.5)


In [125]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_15 (Conv2D)          (None, 94, 94, 32)           896       ['input_11[0][0]']            
                                                                                                  
 max_pooling2d_15 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_15[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 conv2d_16 (Conv2D)          (None, 45, 45, 64)           18496     ['max_pooling2d_15[0][0]

In [126]:
model_dropout.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_18 (Conv2D)          (None, 94, 94, 32)           896       ['input_13[0][0]']            
                                                                                                  
 max_pooling2d_18 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_18[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 dropout_10 (Dropout)        (None, 47, 47, 32)           0         ['max_pooling2d_18[0][0]

In [127]:
from keras.callbacks import ModelCheckpoint

cp = ModelCheckpoint('model_scale/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_scale], y=y_train,
          validation_data=([X_val_pic, X_val_scale], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 4.9309 - accuracy: 0.0134INFO:tensorflow:Assets written to: model_scale\assets


INFO:tensorflow:Assets written to: model_scale\assets


359/359 [==============================] - 129s 353ms/step - loss: 4.9309 - accuracy: 0.0134 - val_loss: 4.6894 - val_accuracy: 0.0253
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.6657 - accuracy: 0.0313INFO:tensorflow:Assets written to: model_scale\assets


INFO:tensorflow:Assets written to: model_scale\assets


359/359 [==============================] - 122s 341ms/step - loss: 4.6657 - accuracy: 0.0313 - val_loss: 4.6337 - val_accuracy: 0.0440
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.6079 - accuracy: 0.0380INFO:tensorflow:Assets written to: model_scale\assets


...

359/359 [==============================] - 129s 359ms/step - loss: 4.4194 - accuracy: 0.0450 - val_loss: 4.4192 - val_accuracy: 0.0505
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 4.4176 - accuracy: 0.0450INFO:tensorflow:Assets written to: model_scale\assets


INFO:tensorflow:Assets written to: model_scale\assets


359/359 [==============================] - 130s 361ms/step - loss: 4.4176 - accuracy: 0.0450 - val_loss: 4.4183 - val_accuracy: 0.0505
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 4.4159 - accuracy: 0.0450INFO:tensorflow:Assets written to: model_scale\assets


INFO:tensorflow:Assets written to: model_scale\assets


359/359 [==============================] - 130s 362ms/step - loss: 4.4159 - accuracy: 0.0450 - val_loss: 4.4177 - val_accuracy: 0.0505


In [128]:
cp = ModelCheckpoint('model_scale_dropout/', save_best_only=True)

model_dropout.fit(x=[X_train_pic, X_train_scale], y=y_train,
          validation_data=([X_val_pic, X_val_scale], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 9.8884 - accuracy: 0.0141INFO:tensorflow:Assets written to: model_scale_dropout\assets


INFO:tensorflow:Assets written to: model_scale_dropout\assets


359/359 [==============================] - 149s 407ms/step - loss: 9.8884 - accuracy: 0.0141 - val_loss: 4.6950 - val_accuracy: 0.0367
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.6871 - accuracy: 0.0300INFO:tensorflow:Assets written to: model_scale_dropout\assets


INFO:tensorflow:Assets written to: model_scale_dropout\assets


359/359 [==============================] - 147s 410ms/step - loss: 4.6871 - accuracy: 0.0300 - val_loss: 4.6550 - val_accuracy: 0.0407
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.6441 - accuracy: 0.0327INFO:tensorflow:Assets written to: model_scale_dropout\assets


...

359/359 [==============================] - 138s 384ms/step - loss: 4.4592 - accuracy: 0.0416 - val_loss: 4.4378 - val_accuracy: 0.0468
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 4.4595 - accuracy: 0.0395INFO:tensorflow:Assets written to: model_scale_dropout\assets


INFO:tensorflow:Assets written to: model_scale_dropout\assets


359/359 [==============================] - 137s 382ms/step - loss: 4.4595 - accuracy: 0.0395 - val_loss: 4.4359 - val_accuracy: 0.0468
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 4.4499 - accuracy: 0.0411INFO:tensorflow:Assets written to: model_scale_dropout\assets


INFO:tensorflow:Assets written to: model_scale_dropout\assets


359/359 [==============================] - 137s 381ms/step - loss: 4.4499 - accuracy: 0.0411 - val_loss: 4.4339 - val_accuracy: 0.0497


In [129]:
from keras.models import load_model

loaded_model = load_model('model_scale/')

In [130]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_scale])
test_predictions

77/77 [==============================] - 8s 99ms/step


array([[0.00639856, 0.01198969, 0.01593084, ..., 0.01113651, 0.00418924,
        0.00568942],
       [0.00709637, 0.00749269, 0.0125805 , ..., 0.01585977, 0.0040253 ,
        0.00607742],
       [0.00948683, 0.00594458, 0.00814151, ..., 0.02344974, 0.00304475,
        0.01321847],
       ...,
       [0.00948683, 0.00594458, 0.00814151, ..., 0.02344974, 0.00304475,
        0.01321847],
       [0.00639856, 0.01198969, 0.01593084, ..., 0.01113651, 0.00418924,
        0.00568942],
       [0.00709637, 0.00749269, 0.0125805 , ..., 0.01585977, 0.0040253 ,
        0.00607742]], dtype=float32)

In [131]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([98, 48, 80, ..., 80, 98, 48], dtype=int64)

In [132]:
true_labels = test_df['label']

In [133]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.011689264602751177

In [134]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.04234527687296417

In [135]:
# Evaluate the model on the test set
test_loss = model.evaluate(x=[X_test_pic, X_test_scale], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 8s 97ms/step - loss: 4.4087 - accuracy: 0.0423
Test Loss: [4.408695697784424, 0.04234527796506882]


In [136]:
loaded_model = load_model('model_scale_dropout/')

In [137]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_scale])
test_predictions

77/77 [==============================] - 8s 96ms/step


array([[0.0064666 , 0.01180174, 0.01496878, ..., 0.01177431, 0.00374346,
        0.00561674],
       [0.00663626, 0.00831837, 0.01369046, ..., 0.01347694, 0.00378257,
        0.00661955],
       [0.00746012, 0.00866387, 0.01065824, ..., 0.0198401 , 0.00330331,
        0.00981194],
       ...,
       [0.00746012, 0.00866387, 0.01065824, ..., 0.0198401 , 0.00330331,
        0.00981194],
       [0.0064666 , 0.01180174, 0.01496878, ..., 0.01177431, 0.00374346,
        0.00561674],
       [0.00663626, 0.00831837, 0.01369046, ..., 0.01347694, 0.00378257,
        0.00661955]], dtype=float32)

In [138]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([98, 48, 48, ..., 48, 98, 48], dtype=int64)

In [139]:
true_labels = test_df['label']

In [140]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.008541220190215049

In [141]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.04234527687296417

In [142]:
# Evaluate the model on the test set
test_loss = model_dropout.evaluate(x=[X_test_pic, X_test_scale], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 7s 94ms/step - loss: 4.4199 - accuracy: 0.0423
Test Loss: [4.419881343841553, 0.04234527796506882]


**CNN + fitzpatrick_centaur**

In [143]:
df = pd.read_csv('fitzpatrick17k_3')
df

,image_name,image_path,label,fitzpatrick_scale,fitzpatrick_centaur,qc,url,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,drug induced pigmentary changes_0,images_full\drug induced pigmentary changes_0.jpg,drug induced pigmentary changes,3,3,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
1,photodermatoses_0,images_full\photodermatoses_0.jpg,photodermatoses,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
2,dermatofibroma_0,images_full\dermatofibroma_0.jpg,dermatofibroma,2,3,NaN,https://www.dermaamin.com/site/images/clinical...,1,0,0,0,0,0,0,0,0,1,0,0
3,psoriasis_0,images_full\psoriasis_0.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
4,psoriasis_1,images_full\psoriasis_1.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,scleroderma_306,images_full\scleroderma_306.jpg,scleroderma,5,5,NaN,http://atlasdermatologico.com.br/img?imageId=4529,0,0,0,0,1,0,0,0,0,0,0,1
16366,pityriasis rosea_190,images_full\pityriasis rosea_190.jpg,pityriasis rosea,4,5,NaN,http://atlasdermatologico.com.br/img?imageId=5630,0,0,0,0,1,0,0,0,0,0,0,1
16367,myiasis_66,images_full\myiasis_66.jpg,myiasis,3,3,NaN,http://atlasdermatologico.com.br/img?imageId=4609,0,0,0,0,1,0,0,0,0,0,0,1
16368,vitiligo_164,images_full\vitiligo_164.jpg,vitiligo,-1,5,NaN,http://atlasdermatologico.com.br/img?imageId=7930,0,0,0,0,1,0,0,0,0,0,0,1


In [144]:
df = df.drop(['fitzpatrick_scale', 'qc', 'url', 'nine_partition_label_benign dermal', 'nine_partition_label_benign epidermal', 'nine_partition_label_benign melanocyte', 'nine_partition_label_genodermatoses', 'nine_partition_label_inflammatory','nine_partition_label_malignant cutaneous lymphoma', 'nine_partition_label_malignant dermal', 'nine_partition_label_malignant epidermal', 'nine_partition_label_malignant melanoma', 'three_partition_label_benign', 'three_partition_label_malignant', 'three_partition_label_non-neoplastic'], axis=1)

In [145]:
# Replace 'images' with 'images_resized' in the 'image_path' column
df['image_path'] = df['image_path'].str.replace('images_full', 'images_resized_full')

In [146]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Visualize the unique and encoded labels
print("Unique labels:", label_encoder.classes_)
print("Encoded labels:", df['label'].unique())

Unique labels: ['acanthosis nigricans' 'acne' 'acne vulgaris'
 'acquired autoimmune bullous diseaseherpes gestationis'
 'acrodermatitis enteropathica' 'actinic keratosis'
 'allergic contact dermatitis' 'aplasia cutis' 'basal cell carcinoma'
 'basal cell carcinoma morpheiform' 'becker nevus' 'behcets disease'
 'calcinosis cutis' 'cheilitis' 'congenital nevus' 'dariers disease'
 'dermatofibroma' 'dermatomyositis' 'disseminated actinic porokeratosis'
 'drug eruption' 'drug induced pigmentary changes' 'dyshidrotic eczema'
 'eczema' 'ehlers danlos syndrome' 'epidermal nevus'
 'epidermolysis bullosa' 'erythema annulare centrifigum'
 'erythema elevatum diutinum' 'erythema multiforme' 'erythema nodosum'
 'factitial dermatitis' 'fixed eruptions' 'folliculitis' 'fordyce spots'
 'granuloma annulare' 'granuloma pyogenic' 'hailey hailey disease'
 'halo nevus' 'hidradenitis' 'ichthyosis vulgaris'
 'incontinentia pigmenti' 'juvenile xanthogranuloma' 'kaposi sarcoma'
 'keloid' 'keratosis pilaris' 'lan

In [147]:
df = pd.get_dummies(df, columns=['fitzpatrick_centaur'])

In [149]:
npz_paths = []

# Loop over the DataFrame rows
for i, row in df.iterrows():
    # Get the image path and .npz file path
    picture_path = row['image_path']
    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    # Load the image
    pic_bgr_arr = cv2.imread(picture_path)
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    # Extracting Fitzpatrick scale values
    minus_one_scale, ones_cale, two_scale = row['fitzpatrick_centaur_-1'], row['fitzpatrick_centaur_1'], row['fitzpatrick_centaur_2']
    three_scale, four_scale, five_scale, six_scale = row['fitzpatrick_centaur_3'], row['fitzpatrick_centaur_4'], row['fitzpatrick_centaur_5'], row['fitzpatrick_centaur_6']
   
    # Creating an array for Fitzpatrick scale values
    centaur = np.array([minus_one_scale, ones_cale,  two_scale, three_scale, four_scale, five_scale, six_scale])
    
    # Getting the overall label
    label = row['label']

    # Save the data into the .npz file
    np.savez_compressed(npz_path, pic=pic_rgb_arr, centaur=centaur, label=label)

# Add the NPZ paths column to the DataFrame
df['NPZ_Path'] = pd.Series(npz_paths)

In [150]:
centaur_df = df.iloc[:,3:]
centaur_df

,fitzpatrick_centaur_-1,fitzpatrick_centaur_1,fitzpatrick_centaur_2,fitzpatrick_centaur_3,fitzpatrick_centaur_4,fitzpatrick_centaur_5,fitzpatrick_centaur_6,NPZ_Path
0,0,0,0,1,0,0,0,images_resized_full\drug induced pigmentary ch...
1,0,1,0,0,0,0,0,images_resized_full\photodermatoses_0.npz
2,0,0,0,1,0,0,0,images_resized_full\dermatofibroma_0.npz
3,0,1,0,0,0,0,0,images_resized_full\psoriasis_0.npz
4,0,1,0,0,0,0,0,images_resized_full\psoriasis_1.npz
...,...,...,...,...,...,...,...,...
16365,0,0,0,0,0,1,0,images_resized_full\scleroderma_306.npz
16366,0,0,0,0,0,1,0,images_resized_full\pityriasis rosea_190.npz
16367,0,0,0,1,0,0,0,images_resized_full\myiasis_66.npz
16368,0,0,0,0,0,1,0,images_resized_full\vitiligo_164.npz


In [151]:
df = df.drop(columns=df.columns.difference(['image_name', 'label', 'NPZ_Path']))
df

,image_name,label,NPZ_Path
0,drug induced pigmentary changes_0,20,images_resized_full\drug induced pigmentary ch...
1,photodermatoses_0,75,images_resized_full\photodermatoses_0.npz
2,dermatofibroma_0,16,images_resized_full\dermatofibroma_0.npz
3,psoriasis_0,86,images_resized_full\psoriasis_0.npz
4,psoriasis_1,86,images_resized_full\psoriasis_1.npz
...,...,...,...
16365,scleroderma_306,93,images_resized_full\scleroderma_306.npz
16366,pityriasis rosea_190,79,images_resized_full\pityriasis rosea_190.npz
16367,myiasis_66,61,images_resized_full\myiasis_66.npz
16368,vitiligo_164,111,images_resized_full\vitiligo_164.npz


In [152]:
shuffled_df = df.sample(frac=1)

# Calculate the size of each set based on desired proportions
total_samples = len(shuffled_df)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

# Divide the DataFrame into training, validation, and test sets
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:train_size+val_size]
test_df = shuffled_df[train_size+val_size:]

len(train_df), len(val_df), len(test_df)

(11459, 2455, 2456)

In [153]:
def load_npz_data(npz_file):
    """Load data from a .npz file."""
    with np.load(npz_file, allow_pickle=True) as data:
        pic = data['pic']
        centaur = data['centaur']
        label = data['label']
    return pic, centaur, label



def get_X_y(df):
    X_pic, X_centaur = [], []
    y = []

    for name in df['NPZ_Path']:
        loaded_npz = np.load(name)

        pic = loaded_npz['pic']
        X_pic.append(pic)

        centaur = loaded_npz['centaur']
        X_centaur.append(centaur)

        y.append(loaded_npz['label'])

    X_pic, X_centaur = np.array(X_pic), np.array(X_centaur)
    y = np.array(y)

    return (X_pic, X_centaur), y

(X_train_pic, X_train_centaur), y_train = get_X_y(train_df)
(X_train_pic.shape, X_train_centaur.shape), y_train.shape

(((11459, 96, 96, 3), (11459, 7)), (11459,))

In [154]:
(X_val_pic, X_val_centaur), y_val = get_X_y(val_df)
(X_val_pic.shape, X_val_centaur.shape), y_val.shape

(((2455, 96, 96, 3), (2455, 7)), (2455,))

In [155]:
(X_test_pic, X_test_centaur), y_test = get_X_y(test_df)
(X_test_pic.shape, X_test_centaur.shape), y_test.shape

(((2456, 96, 96, 3), (2456, 7)), (2456,))

In [156]:
def build_model(conv_params, dense_params, learning_rate=0.0001, dropout=0):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
    pic_output = x

    
    # Stats stream 
    input_stats = layers.Input(shape=(7,))  
    y = input_stats
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output = y  

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model
model = build_model(conv_params, dense_params)

In [157]:
from tensorflow.keras.layers import Dropout

def build_model_dropout(conv_params, dense_params, learning_rate=0.0001, dropout=0.5):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each convolutional layer
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each dense layer
    pic_output = x

    
    # Stats stream 
    input_stats = layers.Input(shape=(7,))  
    y = input_stats
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output = y  

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model with dropout rate of 0.5
model_dropout = build_model_dropout(conv_params, dense_params, dropout=0.5)


In [158]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_21 (Conv2D)          (None, 94, 94, 32)           896       ['input_15[0][0]']            
                                                                                                  
 max_pooling2d_21 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_21[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 conv2d_22 (Conv2D)          (None, 45, 45, 64)           18496     ['max_pooling2d_21[0][0]

In [159]:
model_dropout.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_17 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_24 (Conv2D)          (None, 94, 94, 32)           896       ['input_17[0][0]']            
                                                                                                  
 max_pooling2d_24 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_24[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 dropout_15 (Dropout)        (None, 47, 47, 32)           0         ['max_pooling2d_24[0][0]

In [160]:
cp = ModelCheckpoint('model_centaur/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_centaur], y=y_train,
          validation_data=([X_val_pic, X_val_centaur], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 5.1442 - accuracy: 0.0329INFO:tensorflow:Assets written to: model_centaur\assets


INFO:tensorflow:Assets written to: model_centaur\assets


359/359 [==============================] - 126s 345ms/step - loss: 5.1442 - accuracy: 0.0329 - val_loss: 4.6907 - val_accuracy: 0.0338
Epoch 2/20
359/359 [==============================] - 123s 342ms/step - loss: 4.6538 - accuracy: 0.0402 - val_loss: 4.7080 - val_accuracy: 0.0301
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.5820 - accuracy: 0.0408INFO:tensorflow:Assets written to: model_centaur\assets


INFO:tensorflow:Assets written to: model_centaur\assets


359/359 [==============================] - 122s 341ms/step - loss: 4.5820 - accuracy: 0.0408 - val_loss: 4.5639 - val_accuracy: 0.0346
Epoch 4/20
359/359 [==============================] - ETA: 0s - loss: 4.5090 - accuracy: 0.0408INFO:tensorflow:Assets written to: model_centaur\assets


...

359/359 [==============================] - 129s 361ms/step - loss: 4.2849 - accuracy: 0.0565 - val_loss: 4.3894 - val_accuracy: 0.0436
Epoch 15/20
359/359 [==============================] - 121s 337ms/step - loss: 4.2576 - accuracy: 0.0620 - val_loss: 4.3912 - val_accuracy: 0.0489
Epoch 16/20
359/359 [==============================] - ETA: 0s - loss: 4.2216 - accuracy: 0.0690INFO:tensorflow:Assets written to: model_centaur\assets


INFO:tensorflow:Assets written to: model_centaur\assets


359/359 [==============================] - 122s 339ms/step - loss: 4.2216 - accuracy: 0.0690 - val_loss: 4.3790 - val_accuracy: 0.0415
Epoch 17/20
359/359 [==============================] - ETA: 0s - loss: 4.1755 - accuracy: 0.0764INFO:tensorflow:Assets written to: model_centaur\assets


INFO:tensorflow:Assets written to: model_centaur\assets


359/359 [==============================] - 124s 345ms/step - loss: 4.1755 - accuracy: 0.0764 - val_loss: 4.3748 - val_accuracy: 0.0485
Epoch 18/20
359/359 [==============================] - 124s 344ms/step - loss: 4.0971 - accuracy: 0.0870 - val_loss: 4.3991 - val_accuracy: 0.0542
Epoch 19/20
359/359 [==============================] - 122s 340ms/step - loss: 3.9774 - accuracy: 0.1137 - val_loss: 4.3966 - val_accuracy: 0.0534
Epoch 20/20
359/359 [==============================] - 122s 340ms/step - loss: 3.8076 - accuracy: 0.1446 - val_loss: 4.4132 - val_accuracy: 0.0692


In [161]:
cp = ModelCheckpoint('model_centaur_dropout/', save_best_only=True)

model_dropout.fit(x=[X_train_pic, X_train_centaur], y=y_train,
          validation_data=([X_val_pic, X_val_centaur], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 9.0868 - accuracy: 0.0113INFO:tensorflow:Assets written to: model_centaur_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_dropout\assets


359/359 [==============================] - 139s 379ms/step - loss: 9.0868 - accuracy: 0.0113 - val_loss: 4.7041 - val_accuracy: 0.0240
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.6815 - accuracy: 0.0251INFO:tensorflow:Assets written to: model_centaur_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_dropout\assets


359/359 [==============================] - 139s 387ms/step - loss: 4.6815 - accuracy: 0.0251 - val_loss: 4.6589 - val_accuracy: 0.0330
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.6308 - accuracy: 0.0360INFO:tensorflow:Assets written to: model_centaur_dropout\assets


...

359/359 [==============================] - 132s 367ms/step - loss: 4.3587 - accuracy: 0.0472 - val_loss: 4.4004 - val_accuracy: 0.0440
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 4.3505 - accuracy: 0.0477INFO:tensorflow:Assets written to: model_centaur_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_dropout\assets


359/359 [==============================] - 132s 369ms/step - loss: 4.3505 - accuracy: 0.0477 - val_loss: 4.3975 - val_accuracy: 0.0440
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 4.3510 - accuracy: 0.0487INFO:tensorflow:Assets written to: model_centaur_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_dropout\assets


359/359 [==============================] - 132s 368ms/step - loss: 4.3510 - accuracy: 0.0487 - val_loss: 4.3950 - val_accuracy: 0.0440


In [162]:
loaded_model = load_model('model_centaur/')

In [163]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_centaur])
test_predictions

77/77 [==============================] - 7s 93ms/step


array([[0.00376126, 0.01373452, 0.02897111, ..., 0.00363574, 0.00343226,
        0.00154104],
       [0.01692525, 0.0088812 , 0.01426152, ..., 0.010109  , 0.00257106,
        0.00731897],
       [0.00373477, 0.01385477, 0.02893103, ..., 0.0035737 , 0.00343313,
        0.00154189],
       ...,
       [0.00373477, 0.01385477, 0.02893103, ..., 0.0035737 , 0.00343313,
        0.00154189],
       [0.00377683, 0.01366252, 0.02899286, ..., 0.00367287, 0.0034315 ,
        0.00154041],
       [0.00375755, 0.02006399, 0.03578444, ..., 0.00470001, 0.00386273,
        0.00181406]], dtype=float32)

In [164]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([86, 48, 86, ..., 86, 86, 98], dtype=int64)

In [165]:
true_labels = test_df['label']

In [166]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.027346272368900502

In [167]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.052931596091205214

In [168]:
# Evaluate the model on the test set
test_loss = model.evaluate(x=[X_test_pic, X_test_centaur], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 7s 93ms/step - loss: 4.4558 - accuracy: 0.0753
Test Loss: [4.455794811248779, 0.07532573491334915]


In [169]:
loaded_model = load_model('model_centaur_dropout/')

In [170]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_centaur])
test_predictions

77/77 [==============================] - 8s 96ms/step


array([[0.0038987 , 0.01316058, 0.02513711, ..., 0.00509797, 0.00355477,
        0.00223657],
       [0.00859814, 0.00757313, 0.01669062, ..., 0.01319355, 0.00334373,
        0.00497388],
       [0.0038987 , 0.01316058, 0.02513711, ..., 0.00509797, 0.00355477,
        0.00223657],
       ...,
       [0.0038987 , 0.01316058, 0.02513711, ..., 0.00509797, 0.00355477,
        0.00223657],
       [0.0038987 , 0.01316058, 0.02513711, ..., 0.00509797, 0.00355477,
        0.00223657],
       [0.0038987 , 0.01316058, 0.02513711, ..., 0.00509797, 0.00355477,
        0.00223657]], dtype=float32)

In [171]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([86, 98, 86, ..., 86, 86, 86], dtype=int64)

In [172]:
true_labels = test_df['label']

In [173]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.008064450556625725

In [174]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.04356677524429967

In [175]:
# Evaluate the model on the test set
test_loss = model_dropout.evaluate(x=[X_test_pic, X_test_centaur], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 8s 97ms/step - loss: 4.4195 - accuracy: 0.0436
Test Loss: [4.419456958770752, 0.04356677457690239]


**CNN + nine_partition + three_partition**

In [176]:
df = pd.read_csv('fitzpatrick17k_3')
df

,image_name,image_path,label,fitzpatrick_scale,fitzpatrick_centaur,qc,url,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,drug induced pigmentary changes_0,images_full\drug induced pigmentary changes_0.jpg,drug induced pigmentary changes,3,3,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
1,photodermatoses_0,images_full\photodermatoses_0.jpg,photodermatoses,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
2,dermatofibroma_0,images_full\dermatofibroma_0.jpg,dermatofibroma,2,3,NaN,https://www.dermaamin.com/site/images/clinical...,1,0,0,0,0,0,0,0,0,1,0,0
3,psoriasis_0,images_full\psoriasis_0.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
4,psoriasis_1,images_full\psoriasis_1.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,scleroderma_306,images_full\scleroderma_306.jpg,scleroderma,5,5,NaN,http://atlasdermatologico.com.br/img?imageId=4529,0,0,0,0,1,0,0,0,0,0,0,1
16366,pityriasis rosea_190,images_full\pityriasis rosea_190.jpg,pityriasis rosea,4,5,NaN,http://atlasdermatologico.com.br/img?imageId=5630,0,0,0,0,1,0,0,0,0,0,0,1
16367,myiasis_66,images_full\myiasis_66.jpg,myiasis,3,3,NaN,http://atlasdermatologico.com.br/img?imageId=4609,0,0,0,0,1,0,0,0,0,0,0,1
16368,vitiligo_164,images_full\vitiligo_164.jpg,vitiligo,-1,5,NaN,http://atlasdermatologico.com.br/img?imageId=7930,0,0,0,0,1,0,0,0,0,0,0,1


In [177]:
df = df.drop(['fitzpatrick_scale', 'fitzpatrick_centaur','qc', 'url'], axis=1)

In [178]:
# Replace 'images' with 'images_resized' in the 'image_path' column
df['image_path'] = df['image_path'].str.replace('images_full', 'images_resized_full')

In [179]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Visualize the unique and encoded labels
print("Unique labels:", label_encoder.classes_)
print("Encoded labels:", df['label'].unique())

Unique labels: ['acanthosis nigricans' 'acne' 'acne vulgaris'
 'acquired autoimmune bullous diseaseherpes gestationis'
 'acrodermatitis enteropathica' 'actinic keratosis'
 'allergic contact dermatitis' 'aplasia cutis' 'basal cell carcinoma'
 'basal cell carcinoma morpheiform' 'becker nevus' 'behcets disease'
 'calcinosis cutis' 'cheilitis' 'congenital nevus' 'dariers disease'
 'dermatofibroma' 'dermatomyositis' 'disseminated actinic porokeratosis'
 'drug eruption' 'drug induced pigmentary changes' 'dyshidrotic eczema'
 'eczema' 'ehlers danlos syndrome' 'epidermal nevus'
 'epidermolysis bullosa' 'erythema annulare centrifigum'
 'erythema elevatum diutinum' 'erythema multiforme' 'erythema nodosum'
 'factitial dermatitis' 'fixed eruptions' 'folliculitis' 'fordyce spots'
 'granuloma annulare' 'granuloma pyogenic' 'hailey hailey disease'
 'halo nevus' 'hidradenitis' 'ichthyosis vulgaris'
 'incontinentia pigmenti' 'juvenile xanthogranuloma' 'kaposi sarcoma'
 'keloid' 'keratosis pilaris' 'lan

In [180]:
df

,image_name,image_path,label,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,drug induced pigmentary changes_0,images_resized_full\drug induced pigmentary ch...,20,0,0,0,0,1,0,0,0,0,0,0,1
1,photodermatoses_0,images_resized_full\photodermatoses_0.jpg,75,0,0,0,0,1,0,0,0,0,0,0,1
2,dermatofibroma_0,images_resized_full\dermatofibroma_0.jpg,16,1,0,0,0,0,0,0,0,0,1,0,0
3,psoriasis_0,images_resized_full\psoriasis_0.jpg,86,0,0,0,0,1,0,0,0,0,0,0,1
4,psoriasis_1,images_resized_full\psoriasis_1.jpg,86,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,scleroderma_306,images_resized_full\scleroderma_306.jpg,93,0,0,0,0,1,0,0,0,0,0,0,1
16366,pityriasis rosea_190,images_resized_full\pityriasis rosea_190.jpg,79,0,0,0,0,1,0,0,0,0,0,0,1
16367,myiasis_66,images_resized_full\myiasis_66.jpg,61,0,0,0,0,1,0,0,0,0,0,0,1
16368,vitiligo_164,images_resized_full\vitiligo_164.jpg,111,0,0,0,0,1,0,0,0,0,0,0,1


In [181]:
npz_paths = []

# Loop over the DataFrame rows
for i, row in df.iterrows():
    # Get the image path and .npz file path
    picture_path = row['image_path']
    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    # Load the image
    pic_bgr_arr = cv2.imread(picture_path)
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    # Extracting nine_partition and three_partition labels
    ben_derm, ben_epiderm, ben_mel = row['nine_partition_label_benign dermal'], row['nine_partition_label_benign epidermal'], row['nine_partition_label_benign melanocyte']
    gen, inf = row['nine_partition_label_genodermatoses'], row['nine_partition_label_inflammatory']
    mal_cut, mal_der, mal_epider, mal_mel = row['nine_partition_label_malignant cutaneous lymphoma'], row['nine_partition_label_malignant dermal'], row['nine_partition_label_malignant epidermal'], row['nine_partition_label_malignant melanoma']
    ben, mal, non_neo = row['three_partition_label_benign'], row['three_partition_label_malignant'], row['three_partition_label_non-neoplastic']
    
    # Creating arrays for nine_partition and three_partition labels
    nines = np.array([ben_derm, ben_epiderm, ben_mel, gen, inf, mal_cut, mal_der, mal_epider, mal_mel])
    threes = np.array([ben, mal, non_neo])
    
    # Getting the overall label
    label = row['label']

    # Save the data into the .npz file
    np.savez_compressed(npz_path, pic=pic_rgb_arr, nines=nines, threes=threes, label=label)

# Add the NPZ paths column to the DataFrame
df['NPZ_Path'] = pd.Series(npz_paths)

In [182]:
df

,image_name,image_path,label,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic,NPZ_Path
0,drug induced pigmentary changes_0,images_resized_full\drug induced pigmentary ch...,20,0,0,0,0,1,0,0,0,0,0,0,1,images_resized_full\drug induced pigmentary ch...
1,photodermatoses_0,images_resized_full\photodermatoses_0.jpg,75,0,0,0,0,1,0,0,0,0,0,0,1,images_resized_full\photodermatoses_0.npz
2,dermatofibroma_0,images_resized_full\dermatofibroma_0.jpg,16,1,0,0,0,0,0,0,0,0,1,0,0,images_resized_full\dermatofibroma_0.npz
3,psoriasis_0,images_resized_full\psoriasis_0.jpg,86,0,0,0,0,1,0,0,0,0,0,0,1,images_resized_full\psoriasis_0.npz
4,psoriasis_1,images_resized_full\psoriasis_1.jpg,86,0,0,0,0,1,0,0,0,0,0,0,1,images_resized_full\psoriasis_1.npz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,scleroderma_306,images_resized_full\scleroderma_306.jpg,93,0,0,0,0,1,0,0,0,0,0,0,1,images_resized_full\scleroderma_306.npz
16366,pityriasis rosea_190,images_resized_full\pityriasis rosea_190.jpg,79,0,0,0,0,1,0,0,0,0,0,0,1,images_resized_full\pityriasis rosea_190.npz
16367,myiasis_66,images_resized_full\myiasis_66.jpg,61,0,0,0,0,1,0,0,0,0,0,0,1,images_resized_full\myiasis_66.npz
16368,vitiligo_164,images_resized_full\vitiligo_164.jpg,111,0,0,0,0,1,0,0,0,0,0,0,1,images_resized_full\vitiligo_164.npz


In [183]:
nines_df = df.iloc[:,3:12]
nines_df

,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma
0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
16365,0,0,0,0,1,0,0,0,0
16366,0,0,0,0,1,0,0,0,0
16367,0,0,0,0,1,0,0,0,0
16368,0,0,0,0,1,0,0,0,0


In [184]:
threes_df = df.iloc[:,12:15]
threes_df

,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,0,0,1
1,0,0,1
2,1,0,0
3,0,0,1
4,0,0,1
...,...,...,...
16365,0,0,1
16366,0,0,1
16367,0,0,1
16368,0,0,1


In [185]:
df = df.drop(columns=df.columns.difference(['image_name', 'label', 'NPZ_Path']))
df

,image_name,label,NPZ_Path
0,drug induced pigmentary changes_0,20,images_resized_full\drug induced pigmentary ch...
1,photodermatoses_0,75,images_resized_full\photodermatoses_0.npz
2,dermatofibroma_0,16,images_resized_full\dermatofibroma_0.npz
3,psoriasis_0,86,images_resized_full\psoriasis_0.npz
4,psoriasis_1,86,images_resized_full\psoriasis_1.npz
...,...,...,...
16365,scleroderma_306,93,images_resized_full\scleroderma_306.npz
16366,pityriasis rosea_190,79,images_resized_full\pityriasis rosea_190.npz
16367,myiasis_66,61,images_resized_full\myiasis_66.npz
16368,vitiligo_164,111,images_resized_full\vitiligo_164.npz


In [186]:
shuffled_df = df.sample(frac=1)

# Calculate the size of each set based on desired proportions
total_samples = len(shuffled_df)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

# Divide the DataFrame into training, validation, and test sets
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:train_size+val_size]
test_df = shuffled_df[train_size+val_size:]

len(train_df), len(val_df), len(test_df)

(11459, 2455, 2456)

In [187]:
def load_npz_data(npz_file):
    """Load data from a .npz file."""
    with np.load(npz_file, allow_pickle=True) as data:
        pic = data['pic']
        nines = data['nines']
        threes = data['threes']
        label = data['label']
    return pic, nines, threes, label



def get_X_y(df):
    X_pic, X_nines, X_threes= [], [], []
    y = []

    for name in df['NPZ_Path']:
        loaded_npz = np.load(name)

        pic = loaded_npz['pic']
        X_pic.append(pic)
        
        nines = loaded_npz['nines']
        X_nines.append(nines)
        
        threes = loaded_npz['threes']
        X_threes.append(threes)

        y.append(loaded_npz['label'])

    X_pic, X_nines, X_threes = np.array(X_pic), np.array(X_nines), np.array(X_threes)
    y = np.array(y)

    return (X_pic, X_nines, X_threes), y

(X_train_pic, X_train_nines, X_train_threes), y_train = get_X_y(train_df)
(X_train_pic.shape, X_train_nines.shape, X_train_threes.shape), y_train.shape

(((11459, 96, 96, 3), (11459, 9), (11459, 3)), (11459,))

In [188]:
(X_val_pic, X_val_nines, X_val_threes), y_val = get_X_y(val_df)
(X_val_pic.shape, X_val_nines.shape, X_val_threes.shape), y_val.shape

(((2455, 96, 96, 3), (2455, 9), (2455, 3)), (2455,))

In [189]:
(X_test_pic, X_test_nines, X_test_threes), y_test = get_X_y(test_df)
(X_test_pic.shape, X_test_nines.shape, X_test_threes.shape), y_test.shape

(((2456, 96, 96, 3), (2456, 9), (2456, 3)), (2456,))

In [190]:
def build_model(conv_params, dense_params, learning_rate=0.0001, dropout=0):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(9,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output1 = y
    
     # Stats stream 2
    input_stats2 = layers.Input(shape=(3,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output2 = y

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model
model = build_model(conv_params, dense_params)

In [191]:
from tensorflow.keras.layers import Dropout

def build_model_dropout(conv_params, dense_params, learning_rate=0.0001, dropout=0.5):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each convolutional layer
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each dense layer
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(9,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output1 = y 
    
    # Stats stream 2
    input_stats2 = layers.Input(shape=(3,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output2 = y 

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model with dropout rate of 0.5
model_dropout = build_model_dropout(conv_params, dense_params, dropout=0.5)


In [192]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_19 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_27 (Conv2D)          (None, 94, 94, 32)           896       ['input_19[0][0]']            
                                                                                                  
 max_pooling2d_27 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_27[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 conv2d_28 (Conv2D)          (None, 45, 45, 64)           18496     ['max_pooling2d_27[0][0]

In [193]:
model_dropout.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_22 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_30 (Conv2D)          (None, 94, 94, 32)           896       ['input_22[0][0]']            
                                                                                                  
 max_pooling2d_30 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_30[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 dropout_20 (Dropout)        (None, 47, 47, 32)           0         ['max_pooling2d_30[0][0

In [194]:
cp = ModelCheckpoint('model_nines_threes/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_nines, X_train_threes], y=y_train,
          validation_data=([X_val_pic, X_val_nines, X_val_threes], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 5.3424 - accuracy: 0.0652INFO:tensorflow:Assets written to: model_nines_threes\assets


INFO:tensorflow:Assets written to: model_nines_threes\assets


359/359 [==============================] - 153s 416ms/step - loss: 5.3424 - accuracy: 0.0652 - val_loss: 4.4673 - val_accuracy: 0.1112
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.2909 - accuracy: 0.1226INFO:tensorflow:Assets written to: model_nines_threes\assets


INFO:tensorflow:Assets written to: model_nines_threes\assets


359/359 [==============================] - 155s 433ms/step - loss: 4.2909 - accuracy: 0.1226 - val_loss: 4.0880 - val_accuracy: 0.1295
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 3.9083 - accuracy: 0.1316INFO:tensorflow:Assets written to: model_nines_threes\assets


...

359/359 [==============================] - 147s 410ms/step - loss: 3.2290 - accuracy: 0.1476 - val_loss: 3.2154 - val_accuracy: 0.1511
Epoch 17/20
359/359 [==============================] - ETA: 0s - loss: 3.2261 - accuracy: 0.1478INFO:tensorflow:Assets written to: model_nines_threes\assets


INFO:tensorflow:Assets written to: model_nines_threes\assets


359/359 [==============================] - 157s 436ms/step - loss: 3.2261 - accuracy: 0.1478 - val_loss: 3.2139 - val_accuracy: 0.1507
Epoch 18/20
359/359 [==============================] - 155s 433ms/step - loss: 3.2247 - accuracy: 0.1481 - val_loss: 3.2207 - val_accuracy: 0.1503
Epoch 19/20
359/359 [==============================] - 145s 404ms/step - loss: 3.2232 - accuracy: 0.1487 - val_loss: 3.2150 - val_accuracy: 0.1507
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 3.2235 - accuracy: 0.1485INFO:tensorflow:Assets written to: model_nines_threes\assets


INFO:tensorflow:Assets written to: model_nines_threes\assets


359/359 [==============================] - 111s 310ms/step - loss: 3.2235 - accuracy: 0.1485 - val_loss: 3.2120 - val_accuracy: 0.1507


In [195]:
cp = ModelCheckpoint('model_nines_threes_dropout/', save_best_only=True)

model_dropout.fit(x=[X_train_pic, X_train_nines, X_train_threes], y=y_train,
          validation_data=([X_val_pic, X_val_nines, X_val_threes], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 7.5056 - accuracy: 0.0506INFO:tensorflow:Assets written to: model_nines_threes_dropout\assets


INFO:tensorflow:Assets written to: model_nines_threes_dropout\assets


359/359 [==============================] - 155s 424ms/step - loss: 7.5056 - accuracy: 0.0506 - val_loss: 4.5131 - val_accuracy: 0.0815
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.3864 - accuracy: 0.0955INFO:tensorflow:Assets written to: model_nines_threes_dropout\assets


INFO:tensorflow:Assets written to: model_nines_threes_dropout\assets


359/359 [==============================] - 140s 390ms/step - loss: 4.3864 - accuracy: 0.0955 - val_loss: 4.2076 - val_accuracy: 0.1218
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.0633 - accuracy: 0.1131INFO:tensorflow:Assets written to: model_nines_threes_dropout\assets


...

359/359 [==============================] - 138s 384ms/step - loss: 3.2545 - accuracy: 0.1424 - val_loss: 3.2153 - val_accuracy: 0.1511
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 3.2454 - accuracy: 0.1420INFO:tensorflow:Assets written to: model_nines_threes_dropout\assets


INFO:tensorflow:Assets written to: model_nines_threes_dropout\assets


359/359 [==============================] - 134s 373ms/step - loss: 3.2454 - accuracy: 0.1420 - val_loss: 3.2140 - val_accuracy: 0.1511
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 3.2518 - accuracy: 0.1414INFO:tensorflow:Assets written to: model_nines_threes_dropout\assets


INFO:tensorflow:Assets written to: model_nines_threes_dropout\assets


359/359 [==============================] - 132s 368ms/step - loss: 3.2518 - accuracy: 0.1414 - val_loss: 3.2129 - val_accuracy: 0.1511


In [196]:
loaded_model = load_model('model_nines_threes/')

In [197]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_nines, X_test_threes])
test_predictions

77/77 [==============================] - 7s 93ms/step


array([[8.7004295e-03, 1.7050829e-02, 3.1327721e-02, ..., 1.4744161e-02,
        4.4421866e-03, 7.2187722e-05],
       [2.1144404e-05, 3.2969769e-05, 4.0634921e-05, ..., 3.3850039e-05,
        1.6543243e-05, 2.9635816e-05],
       [8.7004295e-03, 1.7050829e-02, 3.1327721e-02, ..., 1.4744161e-02,
        4.4421866e-03, 7.2187722e-05],
       ...,
       [2.1144404e-05, 3.2969769e-05, 4.0634921e-05, ..., 3.3850003e-05,
        1.6543243e-05, 2.9635816e-05],
       [8.7004295e-03, 1.7050829e-02, 3.1327721e-02, ..., 1.4744161e-02,
        4.4421866e-03, 7.2187722e-05],
       [8.7004295e-03, 1.7050829e-02, 3.1327721e-02, ..., 1.4744161e-02,
        4.4421866e-03, 7.2187722e-05]], dtype=float32)

In [198]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([ 86, 104,  86, ..., 104,  86,  86], dtype=int64)

In [199]:
true_labels = test_df['label']

In [200]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.060601411698043166

In [201]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.13477198697068404

In [202]:
# Evaluate the model on the test set
test_loss = model.evaluate(x=[X_test_pic, X_test_nines, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 7s 95ms/step - loss: 3.2389 - accuracy: 0.1348
Test Loss: [3.238929271697998, 0.13477198779582977]


In [203]:
loaded_model = load_model('model_nines_threes_dropout/')

In [204]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_nines, X_test_threes])
test_predictions

77/77 [==============================] - 7s 91ms/step


array([[8.6385179e-03, 1.6955044e-02, 3.1479239e-02, ..., 1.4897998e-02,
        4.5137974e-03, 1.0802933e-04],
       [3.3629833e-05, 4.0028193e-05, 3.0097841e-05, ..., 4.1810275e-05,
        2.0437557e-05, 5.8909947e-05],
       [8.6385179e-03, 1.6955044e-02, 3.1479239e-02, ..., 1.4897998e-02,
        4.5137974e-03, 1.0802933e-04],
       ...,
       [3.3629825e-05, 4.0028222e-05, 3.0097837e-05, ..., 4.1810268e-05,
        2.0437574e-05, 5.8909940e-05],
       [8.6385179e-03, 1.6955044e-02, 3.1479243e-02, ..., 1.4897998e-02,
        4.5137983e-03, 1.0802933e-04],
       [8.6385179e-03, 1.6955044e-02, 3.1479243e-02, ..., 1.4897998e-02,
        4.5137983e-03, 1.0802933e-04]], dtype=float32)

In [205]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([ 86, 104,  86, ..., 104,  86,  86], dtype=int64)

In [206]:
true_labels = test_df['label']

In [207]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.060601411698043166

In [208]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.13477198697068404

In [209]:
# Evaluate the model on the test set
test_loss = model_dropout.evaluate(x=[X_test_pic, X_test_nines, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 7s 86ms/step - loss: 3.2407 - accuracy: 0.1348
Test Loss: [3.240741729736328, 0.13477198779582977]


**CNN + fitzpatrick_centaur + fitzpatrick_scale**

In [210]:
df = pd.read_csv('fitzpatrick17k_3')
df

,image_name,image_path,label,fitzpatrick_scale,fitzpatrick_centaur,qc,url,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,drug induced pigmentary changes_0,images_full\drug induced pigmentary changes_0.jpg,drug induced pigmentary changes,3,3,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
1,photodermatoses_0,images_full\photodermatoses_0.jpg,photodermatoses,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
2,dermatofibroma_0,images_full\dermatofibroma_0.jpg,dermatofibroma,2,3,NaN,https://www.dermaamin.com/site/images/clinical...,1,0,0,0,0,0,0,0,0,1,0,0
3,psoriasis_0,images_full\psoriasis_0.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
4,psoriasis_1,images_full\psoriasis_1.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,scleroderma_306,images_full\scleroderma_306.jpg,scleroderma,5,5,NaN,http://atlasdermatologico.com.br/img?imageId=4529,0,0,0,0,1,0,0,0,0,0,0,1
16366,pityriasis rosea_190,images_full\pityriasis rosea_190.jpg,pityriasis rosea,4,5,NaN,http://atlasdermatologico.com.br/img?imageId=5630,0,0,0,0,1,0,0,0,0,0,0,1
16367,myiasis_66,images_full\myiasis_66.jpg,myiasis,3,3,NaN,http://atlasdermatologico.com.br/img?imageId=4609,0,0,0,0,1,0,0,0,0,0,0,1
16368,vitiligo_164,images_full\vitiligo_164.jpg,vitiligo,-1,5,NaN,http://atlasdermatologico.com.br/img?imageId=7930,0,0,0,0,1,0,0,0,0,0,0,1


In [211]:
df = df.drop(['qc', 'url', 'nine_partition_label_benign dermal', 'nine_partition_label_benign epidermal', 'nine_partition_label_benign melanocyte',
             'nine_partition_label_genodermatoses', 'nine_partition_label_inflammatory', 'nine_partition_label_malignant cutaneous lymphoma',
             'nine_partition_label_malignant dermal', 'nine_partition_label_malignant epidermal', 'nine_partition_label_malignant melanoma', 
             'three_partition_label_benign', 'three_partition_label_malignant', 'three_partition_label_non-neoplastic'], axis=1)

In [212]:
# Replace 'images' with 'images_resized' in the 'image_path' column
df['image_path'] = df['image_path'].str.replace('images_full', 'images_resized_full')

In [213]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Visualize the unique and encoded labels
print("Unique labels:", label_encoder.classes_)
print("Encoded labels:", df['label'].unique())

Unique labels: ['acanthosis nigricans' 'acne' 'acne vulgaris'
 'acquired autoimmune bullous diseaseherpes gestationis'
 'acrodermatitis enteropathica' 'actinic keratosis'
 'allergic contact dermatitis' 'aplasia cutis' 'basal cell carcinoma'
 'basal cell carcinoma morpheiform' 'becker nevus' 'behcets disease'
 'calcinosis cutis' 'cheilitis' 'congenital nevus' 'dariers disease'
 'dermatofibroma' 'dermatomyositis' 'disseminated actinic porokeratosis'
 'drug eruption' 'drug induced pigmentary changes' 'dyshidrotic eczema'
 'eczema' 'ehlers danlos syndrome' 'epidermal nevus'
 'epidermolysis bullosa' 'erythema annulare centrifigum'
 'erythema elevatum diutinum' 'erythema multiforme' 'erythema nodosum'
 'factitial dermatitis' 'fixed eruptions' 'folliculitis' 'fordyce spots'
 'granuloma annulare' 'granuloma pyogenic' 'hailey hailey disease'
 'halo nevus' 'hidradenitis' 'ichthyosis vulgaris'
 'incontinentia pigmenti' 'juvenile xanthogranuloma' 'kaposi sarcoma'
 'keloid' 'keratosis pilaris' 'lan

In [214]:
df = pd.get_dummies(df, columns=['fitzpatrick_centaur', 'fitzpatrick_scale'])

In [215]:
npz_paths = []

# Loop over the DataFrame rows
for i, row in df.iterrows():
    # Get the image path and .npz file path
    picture_path = row['image_path']
    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    # Load the image
    pic_bgr_arr = cv2.imread(picture_path)
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    # Extracting Fitzpatrick centaur and scale values
    minus_one, one, two = row['fitzpatrick_centaur_-1'], row['fitzpatrick_centaur_1'], row['fitzpatrick_centaur_2']
    three, four, five, six = row['fitzpatrick_centaur_3'], row['fitzpatrick_centaur_4'], row['fitzpatrick_centaur_5'], row['fitzpatrick_centaur_6']
    
    minus_one_scale, ones_cale, two_scale = row['fitzpatrick_scale_-1'], row['fitzpatrick_scale_1'], row['fitzpatrick_scale_2']
    three_scale, four_scale, five_scale, six_scale = row['fitzpatrick_scale_3'], row['fitzpatrick_scale_4'], row['fitzpatrick_scale_5'], row['fitzpatrick_scale_6']
    
    # Creating arrays for Fitzpatrick centaur and scale values
    centaur = np.array([minus_one, one, two, three, four, five, six])
    scale = np.array([minus_one_scale, ones_cale,  two_scale, three_scale, four_scale, five_scale, six_scale])
    
    # Getting the overall label
    label = row['label']

    # Save the data into the .npz file
    np.savez_compressed(npz_path, pic=pic_rgb_arr, centaur=centaur, scale=scale, label=label)

# Add the NPZ paths column to the DataFrame
df['NPZ_Path'] = pd.Series(npz_paths)

In [216]:
centaur_df = df[['fitzpatrick_centaur_-1', 'fitzpatrick_centaur_1', 'fitzpatrick_centaur_2', 'fitzpatrick_centaur_3', 'fitzpatrick_centaur_4', 'fitzpatrick_centaur_5', 'fitzpatrick_centaur_6']]

In [217]:
scale_df = df[['fitzpatrick_scale_-1', 'fitzpatrick_scale_1', 'fitzpatrick_scale_2', 'fitzpatrick_scale_3', 'fitzpatrick_scale_4', 'fitzpatrick_scale_5', 'fitzpatrick_scale_6']]

In [218]:
df = df.drop(columns=df.columns.difference(['image_name', 'label', 'NPZ_Path']))
df

,image_name,label,NPZ_Path
0,drug induced pigmentary changes_0,20,images_resized_full\drug induced pigmentary ch...
1,photodermatoses_0,75,images_resized_full\photodermatoses_0.npz
2,dermatofibroma_0,16,images_resized_full\dermatofibroma_0.npz
3,psoriasis_0,86,images_resized_full\psoriasis_0.npz
4,psoriasis_1,86,images_resized_full\psoriasis_1.npz
...,...,...,...
16365,scleroderma_306,93,images_resized_full\scleroderma_306.npz
16366,pityriasis rosea_190,79,images_resized_full\pityriasis rosea_190.npz
16367,myiasis_66,61,images_resized_full\myiasis_66.npz
16368,vitiligo_164,111,images_resized_full\vitiligo_164.npz


In [219]:
shuffled_df = df.sample(frac=1)

# Calculate the size of each set based on desired proportions
total_samples = len(shuffled_df)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

# Divide the DataFrame into training, validation, and test sets
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:train_size+val_size]
test_df = shuffled_df[train_size+val_size:]

len(train_df), len(val_df), len(test_df)

(11459, 2455, 2456)

In [220]:
def load_npz_data(npz_file):
    """Load data from a .npz file."""
    with np.load(npz_file, allow_pickle=True) as data:
        pic = data['pic']
        centaur = data['centaur']
        scale = data['scale']
        label = data['label']
    return pic, centaur, scale, label



def get_X_y(df):
    X_pic, X_centaur, X_scale= [], [], []
    y = []

    for name in df['NPZ_Path']:
        loaded_npz = np.load(name)

        pic = loaded_npz['pic']
        X_pic.append(pic)
        
        centaur = loaded_npz['centaur']
        X_centaur.append(centaur)
        
        scale = loaded_npz['scale']
        X_scale.append(scale)

        y.append(loaded_npz['label'])

    X_pic, X_centaur, X_scale = np.array(X_pic), np.array(X_centaur), np.array(X_scale)
    y = np.array(y)

    return (X_pic, X_centaur, X_scale), y

(X_train_pic, X_train_centaur, X_train_scale), y_train = get_X_y(train_df)
(X_train_pic.shape, X_train_centaur.shape, X_train_scale.shape), y_train.shape

(((11459, 96, 96, 3), (11459, 7), (11459, 7)), (11459,))

In [221]:
(X_val_pic, X_val_centaur, X_val_scale), y_val = get_X_y(val_df)
(X_val_pic.shape, X_val_centaur.shape, X_val_scale.shape), y_val.shape

(((2455, 96, 96, 3), (2455, 7), (2455, 7)), (2455,))

In [222]:
(X_test_pic, X_test_centaur, X_test_scale), y_test = get_X_y(test_df)
(X_test_pic.shape, X_test_centaur.shape, X_test_scale.shape), y_test.shape

(((2456, 96, 96, 3), (2456, 7), (2456, 7)), (2456,))

In [223]:
def build_model(conv_params, dense_params, learning_rate=0.0001, dropout=0):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(7,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output1 = y
    
     # Stats stream 2
    input_stats2 = layers.Input(shape=(7,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output2 = y
    

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model
model = build_model(conv_params, dense_params)

In [224]:
from tensorflow.keras.layers import Dropout

def build_model_dropout(conv_params, dense_params, learning_rate=0.0001, dropout=0.5):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each convolutional layer
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each dense layer
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(7,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output1 = y  
    
    # Stats stream 2
    input_stats2 = layers.Input(shape=(7,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output2 = y 

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model with dropout rate of 0.5
model_dropout = build_model_dropout(conv_params, dense_params, dropout=0.5)


In [225]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_25 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_33 (Conv2D)          (None, 94, 94, 32)           896       ['input_25[0][0]']            
                                                                                                  
 max_pooling2d_33 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_33[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 conv2d_34 (Conv2D)          (None, 45, 45, 64)           18496     ['max_pooling2d_33[0][0

In [226]:
model_dropout.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_28 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_36 (Conv2D)          (None, 94, 94, 32)           896       ['input_28[0][0]']            
                                                                                                  
 max_pooling2d_36 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_36[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 dropout_26 (Dropout)        (None, 47, 47, 32)           0         ['max_pooling2d_36[0][0

In [227]:
cp = ModelCheckpoint('model_centaur_scale/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_centaur, X_train_scale], y=y_train,
          validation_data=([X_val_pic, X_val_centaur, X_val_scale], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 5.0343 - accuracy: 0.0183INFO:tensorflow:Assets written to: model_centaur_scale\assets


INFO:tensorflow:Assets written to: model_centaur_scale\assets


359/359 [==============================] - 120s 330ms/step - loss: 5.0343 - accuracy: 0.0183 - val_loss: 4.6534 - val_accuracy: 0.0334
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.6116 - accuracy: 0.0372INFO:tensorflow:Assets written to: model_centaur_scale\assets


INFO:tensorflow:Assets written to: model_centaur_scale\assets


359/359 [==============================] - 121s 338ms/step - loss: 4.6116 - accuracy: 0.0372 - val_loss: 4.5650 - val_accuracy: 0.0440
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.5234 - accuracy: 0.0421INFO:tensorflow:Assets written to: model_centaur_scale\assets


...

359/359 [==============================] - 126s 351ms/step - loss: 4.2955 - accuracy: 0.0528 - val_loss: 4.3461 - val_accuracy: 0.0456
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 4.2899 - accuracy: 0.0538INFO:tensorflow:Assets written to: model_centaur_scale\assets


INFO:tensorflow:Assets written to: model_centaur_scale\assets


359/359 [==============================] - 125s 350ms/step - loss: 4.2899 - accuracy: 0.0538 - val_loss: 4.3454 - val_accuracy: 0.0448
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 4.2870 - accuracy: 0.0531INFO:tensorflow:Assets written to: model_centaur_scale\assets


INFO:tensorflow:Assets written to: model_centaur_scale\assets


359/359 [==============================] - 127s 354ms/step - loss: 4.2870 - accuracy: 0.0531 - val_loss: 4.3448 - val_accuracy: 0.0448


In [228]:
cp = ModelCheckpoint('model_centaur_scale_dropout/', save_best_only=True)

model_dropout.fit(x=[X_train_pic, X_train_centaur, X_train_scale], y=y_train,
          validation_data=([X_val_pic, X_val_centaur, X_val_scale], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 6.6854 - accuracy: 0.0230INFO:tensorflow:Assets written to: model_centaur_scale_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_scale_dropout\assets


359/359 [==============================] - 154s 418ms/step - loss: 6.6854 - accuracy: 0.0230 - val_loss: 4.6632 - val_accuracy: 0.0411
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.6382 - accuracy: 0.0356INFO:tensorflow:Assets written to: model_centaur_scale_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_scale_dropout\assets


359/359 [==============================] - 141s 394ms/step - loss: 4.6382 - accuracy: 0.0356 - val_loss: 4.5968 - val_accuracy: 0.0424
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 4.5730 - accuracy: 0.0348INFO:tensorflow:Assets written to: model_centaur_scale_dropout\assets


...

359/359 [==============================] - 147s 408ms/step - loss: 4.3392 - accuracy: 0.0483 - val_loss: 4.3618 - val_accuracy: 0.0448
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 4.3342 - accuracy: 0.0476INFO:tensorflow:Assets written to: model_centaur_scale_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_scale_dropout\assets


359/359 [==============================] - 138s 385ms/step - loss: 4.3342 - accuracy: 0.0476 - val_loss: 4.3595 - val_accuracy: 0.0481
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 4.3288 - accuracy: 0.0505INFO:tensorflow:Assets written to: model_centaur_scale_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_scale_dropout\assets


359/359 [==============================] - 135s 376ms/step - loss: 4.3288 - accuracy: 0.0505 - val_loss: 4.3575 - val_accuracy: 0.0481


In [229]:
loaded_model = load_model('model_centaur_scale/')

In [230]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_centaur, X_test_scale])
test_predictions

77/77 [==============================] - 7s 93ms/step


array([[0.00296042, 0.0146751 , 0.03864783, ..., 0.00279095, 0.00230518,
        0.00151511],
       [0.01000564, 0.0064599 , 0.00710984, ..., 0.0331611 , 0.00354175,
        0.02073672],
       [0.00541687, 0.01021999, 0.02189984, ..., 0.00581536, 0.00326177,
        0.00236117],
       ...,
       [0.00642783, 0.00804388, 0.01274872, ..., 0.00841665, 0.00381487,
        0.00340137],
       [0.00285303, 0.01560463, 0.02519933, ..., 0.00338034, 0.00178883,
        0.00166879],
       [0.00274308, 0.01382636, 0.04111167, ..., 0.00300042, 0.00301424,
        0.00134783]], dtype=float32)

In [231]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([ 8, 48, 98, ..., 81, 86, 86], dtype=int64)

In [232]:
true_labels = test_df['label']

In [233]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.010713477305309846

In [234]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.039495114006514655

In [235]:
# Evaluate the model on the test set
test_loss = model.evaluate(x=[X_test_pic, X_test_centaur, X_test_scale], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 7s 95ms/step - loss: 4.3798 - accuracy: 0.0395
Test Loss: [4.379788875579834, 0.039495114237070084]


In [236]:
loaded_model = load_model('model_centaur_scale_dropout/')

In [237]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_centaur, X_test_scale])
test_predictions

77/77 [==============================] - 7s 95ms/step


array([[0.00412808, 0.01423957, 0.0344372 , ..., 0.00391166, 0.00247878,
        0.0025237 ],
       [0.00959296, 0.00631915, 0.00693492, ..., 0.03233184, 0.00420035,
        0.0188417 ],
       [0.00561745, 0.01223445, 0.02053815, ..., 0.00829568, 0.00328028,
        0.00270273],
       ...,
       [0.00660634, 0.00948183, 0.01341491, ..., 0.01022385, 0.0033298 ,
        0.00375707],
       [0.00362224, 0.01496995, 0.02503624, ..., 0.00446124, 0.00213592,
        0.00250353],
       [0.00384495, 0.01405646, 0.03571582, ..., 0.00387887, 0.00306828,
        0.0017357 ]], dtype=float32)

In [238]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([98, 48, 98, ..., 98, 86, 86], dtype=int64)

In [239]:
true_labels = test_df['label']

In [240]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.00980036270921

In [241]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.03908794788273615

In [242]:
# Evaluate the model on the test set
test_loss = model_dropout.evaluate(x=[X_test_pic, X_test_centaur, X_test_scale], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 7s 95ms/step - loss: 4.3942 - accuracy: 0.0391
Test Loss: [4.394208908081055, 0.03908794745802879]


**CNN + fitzpatrick_centaur + fitzpatrick_scale + nine_partition + three_partition**

In [243]:
df = pd.read_csv('fitzpatrick17k_3')
df

,image_name,image_path,label,fitzpatrick_scale,fitzpatrick_centaur,qc,url,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,drug induced pigmentary changes_0,images_full\drug induced pigmentary changes_0.jpg,drug induced pigmentary changes,3,3,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
1,photodermatoses_0,images_full\photodermatoses_0.jpg,photodermatoses,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
2,dermatofibroma_0,images_full\dermatofibroma_0.jpg,dermatofibroma,2,3,NaN,https://www.dermaamin.com/site/images/clinical...,1,0,0,0,0,0,0,0,0,1,0,0
3,psoriasis_0,images_full\psoriasis_0.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
4,psoriasis_1,images_full\psoriasis_1.jpg,psoriasis,1,1,NaN,https://www.dermaamin.com/site/images/clinical...,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,scleroderma_306,images_full\scleroderma_306.jpg,scleroderma,5,5,NaN,http://atlasdermatologico.com.br/img?imageId=4529,0,0,0,0,1,0,0,0,0,0,0,1
16366,pityriasis rosea_190,images_full\pityriasis rosea_190.jpg,pityriasis rosea,4,5,NaN,http://atlasdermatologico.com.br/img?imageId=5630,0,0,0,0,1,0,0,0,0,0,0,1
16367,myiasis_66,images_full\myiasis_66.jpg,myiasis,3,3,NaN,http://atlasdermatologico.com.br/img?imageId=4609,0,0,0,0,1,0,0,0,0,0,0,1
16368,vitiligo_164,images_full\vitiligo_164.jpg,vitiligo,-1,5,NaN,http://atlasdermatologico.com.br/img?imageId=7930,0,0,0,0,1,0,0,0,0,0,0,1


In [244]:
df = df.drop(['qc', 'url'], axis=1)

In [245]:
# Replace 'images' with 'images_resized' in the 'image_path' column
df['image_path'] = df['image_path'].str.replace('images_full', 'images_resized_full')

In [246]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Visualize the unique and encoded labels
print("Unique labels:", label_encoder.classes_)
print("Encoded labels:", df['label'].unique())

Unique labels: ['acanthosis nigricans' 'acne' 'acne vulgaris'
 'acquired autoimmune bullous diseaseherpes gestationis'
 'acrodermatitis enteropathica' 'actinic keratosis'
 'allergic contact dermatitis' 'aplasia cutis' 'basal cell carcinoma'
 'basal cell carcinoma morpheiform' 'becker nevus' 'behcets disease'
 'calcinosis cutis' 'cheilitis' 'congenital nevus' 'dariers disease'
 'dermatofibroma' 'dermatomyositis' 'disseminated actinic porokeratosis'
 'drug eruption' 'drug induced pigmentary changes' 'dyshidrotic eczema'
 'eczema' 'ehlers danlos syndrome' 'epidermal nevus'
 'epidermolysis bullosa' 'erythema annulare centrifigum'
 'erythema elevatum diutinum' 'erythema multiforme' 'erythema nodosum'
 'factitial dermatitis' 'fixed eruptions' 'folliculitis' 'fordyce spots'
 'granuloma annulare' 'granuloma pyogenic' 'hailey hailey disease'
 'halo nevus' 'hidradenitis' 'ichthyosis vulgaris'
 'incontinentia pigmenti' 'juvenile xanthogranuloma' 'kaposi sarcoma'
 'keloid' 'keratosis pilaris' 'lan

In [247]:
df = pd.get_dummies(df, columns=['fitzpatrick_centaur', 'fitzpatrick_scale'])

In [248]:
npz_paths = []

# Loop over the DataFrame rows
for i, row in df.iterrows():
    # Get the image path and .npz file path
    picture_path = row['image_path']
    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    # Load the image
    pic_bgr_arr = cv2.imread(picture_path)
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    # Extracting Fitzpatrick centaur, scale, nine_partition, and three_partition labels
    minus_one, one, two = row['fitzpatrick_centaur_-1'], row['fitzpatrick_centaur_1'], row['fitzpatrick_centaur_2']
    three, four, five, six = row['fitzpatrick_centaur_3'], row['fitzpatrick_centaur_4'], row['fitzpatrick_centaur_5'], row['fitzpatrick_centaur_6']
    
    minus_one_scale, ones_cale, two_scale = row['fitzpatrick_scale_-1'], row['fitzpatrick_scale_1'], row['fitzpatrick_scale_2']
    three_scale, four_scale, five_scale, six_scale = row['fitzpatrick_scale_3'], row['fitzpatrick_scale_4'], row['fitzpatrick_scale_5'], row['fitzpatrick_scale_6']
   
    ben_derm, ben_epiderm, ben_mel = row['nine_partition_label_benign dermal'], row['nine_partition_label_benign epidermal'], row['nine_partition_label_benign melanocyte']
    gen, inf = row['nine_partition_label_genodermatoses'], row['nine_partition_label_inflammatory']
    mal_cut, mal_der, mal_epider, mal_mel = row['nine_partition_label_malignant cutaneous lymphoma'], row['nine_partition_label_malignant dermal'], row['nine_partition_label_malignant epidermal'], row['nine_partition_label_malignant melanoma']
    
    ben, mal, non_neo = row['three_partition_label_benign'], row['three_partition_label_malignant'], row['three_partition_label_non-neoplastic']
    
    # Creating arrays for Fitzpatrick centaur, scale, nine_partition, and three_partition labels
    threes = np.array([ben, mal, non_neo])
    nines = np.array([ben_derm, ben_epiderm, ben_mel, gen, inf, mal_cut, mal_der, mal_epider, mal_mel])
    centaur = np.array([minus_one, one, two, three, four, five, six])
    scale = np.array([minus_one_scale, ones_cale,  two_scale, three_scale, four_scale, five_scale, six_scale])
    
    # Getting the overall label
    label = row['label']

    # Save the data into the .npz file
    np.savez_compressed(npz_path, pic=pic_rgb_arr, centaur=centaur, scale=scale, nines=nines, threes=threes, label=label)

# Add the NPZ paths column to the DataFrame
df['NPZ_Path'] = pd.Series(npz_paths)

In [249]:
centaur_df = df[['fitzpatrick_centaur_-1', 'fitzpatrick_centaur_1', 'fitzpatrick_centaur_2', 'fitzpatrick_centaur_3', 'fitzpatrick_centaur_4', 'fitzpatrick_centaur_5', 'fitzpatrick_centaur_6']]

In [250]:
scale_df = df[['fitzpatrick_scale_-1', 'fitzpatrick_scale_1', 'fitzpatrick_scale_2', 'fitzpatrick_scale_3', 'fitzpatrick_scale_4', 'fitzpatrick_scale_5', 'fitzpatrick_scale_6']]

In [251]:
# Creating a list of features starting with 'nine_partition'
nine_features = [col for col in df.columns if col.startswith('nine_partition')]

# Creating a list of features starting with 'three_partition'
three_features = [col for col in df.columns if col.startswith('three_partition')]

# Creating the DataFrame nines_df with features starting with 'nine_partition'
nines_df = df[nine_features]

# Creating the DataFrame threes_df with features starting with 'three_partition'
threes_df = df[three_features]

In [252]:
nines_df

,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma
0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
16365,0,0,0,0,1,0,0,0,0
16366,0,0,0,0,1,0,0,0,0
16367,0,0,0,0,1,0,0,0,0
16368,0,0,0,0,1,0,0,0,0


In [253]:
threes_df 

,three_partition_label_benign,three_partition_label_malignant,three_partition_label_non-neoplastic
0,0,0,1
1,0,0,1
2,1,0,0
3,0,0,1
4,0,0,1
...,...,...,...
16365,0,0,1
16366,0,0,1
16367,0,0,1
16368,0,0,1


In [254]:
df = df.drop(columns=df.columns.difference(['image_name', 'label', 'NPZ_Path']))
df

,image_name,label,NPZ_Path
0,drug induced pigmentary changes_0,20,images_resized_full\drug induced pigmentary ch...
1,photodermatoses_0,75,images_resized_full\photodermatoses_0.npz
2,dermatofibroma_0,16,images_resized_full\dermatofibroma_0.npz
3,psoriasis_0,86,images_resized_full\psoriasis_0.npz
4,psoriasis_1,86,images_resized_full\psoriasis_1.npz
...,...,...,...
16365,scleroderma_306,93,images_resized_full\scleroderma_306.npz
16366,pityriasis rosea_190,79,images_resized_full\pityriasis rosea_190.npz
16367,myiasis_66,61,images_resized_full\myiasis_66.npz
16368,vitiligo_164,111,images_resized_full\vitiligo_164.npz


In [255]:
shuffled_df = df.sample(frac=1)

# Calculate the size of each set based on desired proportions
total_samples = len(shuffled_df)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

# Divide the DataFrame into training, validation, and test sets
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:train_size+val_size]
test_df = shuffled_df[train_size+val_size:]

len(train_df), len(val_df), len(test_df)

(11459, 2455, 2456)

In [256]:
def load_npz_data(npz_file):
    """Load data from a .npz file."""
    with np.load(npz_file, allow_pickle=True) as data:
        pic = data['pic']
        centaur = data['centaur']
        scale = data['scale']
        nines = data['nines']
        threes = data['threes']
        label = data['label']
    return pic, centaur, scale, nines, threes, label



def get_X_y(df):
    X_pic, X_centaur, X_scale, X_nines, X_threes= [], [], [], [], []
    y = []

    for name in df['NPZ_Path']:
        loaded_npz = np.load(name)

        pic = loaded_npz['pic']
        X_pic.append(pic)
        
        centaur = loaded_npz['centaur']
        X_centaur.append(centaur)
        
        scale = loaded_npz['scale']
        X_scale.append(scale)
        
        nines = loaded_npz['nines']
        X_nines.append(nines)
        
        threes = loaded_npz['threes']
        X_threes.append(threes)

        y.append(loaded_npz['label'])

    X_pic, X_centaur, X_scale, X_nines, X_threes = np.array(X_pic), np.array(X_centaur), np.array(X_scale), np.array(X_nines), np.array(X_threes)
    y = np.array(y)

    return (X_pic, X_centaur, X_scale, X_nines, X_threes), y

(X_train_pic, X_train_centaur, X_train_scale, X_train_nines, X_train_threes), y_train = get_X_y(train_df)
(X_train_pic.shape, X_train_centaur.shape, X_train_scale.shape, X_train_nines.shape, X_train_threes.shape), y_train.shape

(((11459, 96, 96, 3), (11459, 7), (11459, 7), (11459, 9), (11459, 3)),
 (11459,))

In [257]:
(X_val_pic, X_val_centaur, X_val_scale, X_val_nines, X_val_threes), y_val = get_X_y(val_df)
(X_val_pic.shape, X_val_centaur.shape, X_val_scale.shape, X_val_nines.shape, X_val_threes.shape), y_val.shape

(((2455, 96, 96, 3), (2455, 7), (2455, 7), (2455, 9), (2455, 3)), (2455,))

In [260]:
(X_test_pic, X_test_centaur, X_test_scale, X_test_nines, X_test_threes), y_test = get_X_y(test_df)
(X_test_pic.shape, X_test_centaur.shape, X_test_scale.shape, X_test_nines.shape, X_test_threes.shape), y_test.shape

(((2456, 96, 96, 3), (2456, 7), (2456, 7), (2456, 9), (2456, 3)), (2456,))

In [268]:
def build_model(conv_params, dense_params, learning_rate=0.0001, dropout=0):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(7,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output1 = y
    
     # Stats stream 2
    input_stats2 = layers.Input(shape=(7,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output2 = y
    
      # Stats stream 3
    input_stats3 = layers.Input(shape=(9,))  
    y = input_stats3
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output3 = y
    
      # Stats stream 4
    input_stats4 = layers.Input(shape=(3,))  
    y = input_stats4
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output4 = y

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2, stats_output3, stats_output4])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2, input_stats3, input_stats4], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model
model = build_model(conv_params, dense_params)

In [269]:
from tensorflow.keras.layers import Dropout

def build_model_dropout(conv_params, dense_params, learning_rate=0.0001, dropout=0.5):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each convolutional layer
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each dense layer
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(7,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output1 = y
    
    # Stats stream 2
    input_stats2 = layers.Input(shape=(7,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output2 = y 
    
    # Stats stream 3
    input_stats3 = layers.Input(shape=(9,))  
    y = input_stats3
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output3 = y 
    
    # Stats stream 4
    input_stats4 = layers.Input(shape=(3,))  
    y = input_stats4
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output4 = y 

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2, stats_output3, stats_output4])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2, input_stats3, input_stats4], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model with dropout rate of 0.5
model_dropout = build_model_dropout(conv_params, dense_params, dropout=0.5)


In [270]:
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_41 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_45 (Conv2D)          (None, 94, 94, 32)           896       ['input_41[0][0]']            
                                                                                                  
 max_pooling2d_45 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_45[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 conv2d_46 (Conv2D)          (None, 45, 45, 64)           18496     ['max_pooling2d_45[0][0

In [271]:
model_dropout.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_46 (InputLayer)       [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 conv2d_48 (Conv2D)          (None, 94, 94, 32)           896       ['input_46[0][0]']            
                                                                                                  
 max_pooling2d_48 (MaxPooli  (None, 47, 47, 32)           0         ['conv2d_48[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 dropout_40 (Dropout)        (None, 47, 47, 32)           0         ['max_pooling2d_48[0][0

In [272]:
cp = ModelCheckpoint('model_centaur_scale_nines_threes/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_centaur, X_train_scale, X_train_nines, X_train_threes], y=y_train,
          validation_data=([X_val_pic, X_val_centaur, X_val_scale, X_val_nines, X_val_threes], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 4.7376 - accuracy: 0.0567INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes\assets


INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes\assets


359/359 [==============================] - 103s 282ms/step - loss: 4.7376 - accuracy: 0.0567 - val_loss: 4.4078 - val_accuracy: 0.0733
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.2104 - accuracy: 0.0849INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes\assets


INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes\assets


359/359 [==============================] - 118s 330ms/step - loss: 4.2104 - accuracy: 0.0849 - val_loss: 3.9992 - val_accuracy: 0.1010
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 3.8193 - accuracy: 0.1312INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes\assets


...

359/359 [==============================] - 134s 372ms/step - loss: 3.0544 - accuracy: 0.1863 - val_loss: 3.0877 - val_accuracy: 0.1654
Epoch 14/20
359/359 [==============================] - ETA: 0s - loss: 3.0195 - accuracy: 0.1945INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes\assets


INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes\assets


359/359 [==============================] - 130s 363ms/step - loss: 3.0195 - accuracy: 0.1945 - val_loss: 3.0770 - val_accuracy: 0.1715
Epoch 15/20
359/359 [==============================] - 129s 359ms/step - loss: 2.9636 - accuracy: 0.2096 - val_loss: 3.0810 - val_accuracy: 0.1862
Epoch 16/20
359/359 [==============================] - ETA: 0s - loss: 2.8846 - accuracy: 0.2272INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes\assets


INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes\assets


359/359 [==============================] - 128s 357ms/step - loss: 2.8846 - accuracy: 0.2272 - val_loss: 3.0770 - val_accuracy: 0.1874
Epoch 17/20
359/359 [==============================] - 129s 360ms/step - loss: 2.7486 - accuracy: 0.2655 - val_loss: 3.0813 - val_accuracy: 0.1980
Epoch 18/20
359/359 [==============================] - 126s 351ms/step - loss: 2.5461 - accuracy: 0.3203 - val_loss: 3.1322 - val_accuracy: 0.2069
Epoch 19/20
359/359 [==============================] - 123s 343ms/step - loss: 2.2750 - accuracy: 0.3856 - val_loss: 3.1774 - val_accuracy: 0.2155
Epoch 20/20
359/359 [==============================] - 123s 342ms/step - loss: 1.9688 - accuracy: 0.4735 - val_loss: 3.4490 - val_accuracy: 0.2244


In [273]:
cp = ModelCheckpoint('model_centaur_scale_nines_threes_dropout/', save_best_only=True)

model_dropout.fit(x=[X_train_pic, X_train_centaur, X_train_scale, X_train_nines, X_train_threes], y=y_train,
          validation_data=([X_val_pic, X_val_centaur, X_val_scale, X_val_nines, X_val_threes], y_val),
          epochs=20, 
          callbacks=[cp])

Epoch 1/20
359/359 [==============================] - ETA: 0s - loss: 6.8391 - accuracy: 0.0521INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes_dropout\assets


359/359 [==============================] - 145s 399ms/step - loss: 6.8391 - accuracy: 0.0521 - val_loss: 4.4430 - val_accuracy: 0.0933
Epoch 2/20
359/359 [==============================] - ETA: 0s - loss: 4.2831 - accuracy: 0.1011INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes_dropout\assets


359/359 [==============================] - 146s 407ms/step - loss: 4.2831 - accuracy: 0.1011 - val_loss: 4.0899 - val_accuracy: 0.1022
Epoch 3/20
359/359 [==============================] - ETA: 0s - loss: 3.9322 - accuracy: 0.1181INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes_dropout\assets


...

359/359 [==============================] - 145s 403ms/step - loss: 3.0986 - accuracy: 0.1735 - val_loss: 3.1008 - val_accuracy: 0.1597
Epoch 19/20
359/359 [==============================] - ETA: 0s - loss: 3.0986 - accuracy: 0.1739INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes_dropout\assets


359/359 [==============================] - 148s 414ms/step - loss: 3.0986 - accuracy: 0.1739 - val_loss: 3.0970 - val_accuracy: 0.1601
Epoch 20/20
359/359 [==============================] - ETA: 0s - loss: 3.0916 - accuracy: 0.1758INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes_dropout\assets


INFO:tensorflow:Assets written to: model_centaur_scale_nines_threes_dropout\assets


359/359 [==============================] - 146s 408ms/step - loss: 3.0916 - accuracy: 0.1758 - val_loss: 3.0945 - val_accuracy: 0.1613


In [274]:
loaded_model = load_model('model_centaur_scale_nines_threes/')

In [275]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_centaur, X_test_scale, X_test_nines, X_test_threes])
test_predictions

77/77 [==============================] - 8s 96ms/step


array([[1.1971388e-06, 9.9192303e-06, 1.0478749e-06, ..., 6.1310334e-07,
        1.5157747e-06, 2.7038852e-06],
       [5.1504718e-03, 2.6331402e-03, 1.2396567e-02, ..., 1.8319702e-02,
        3.2219298e-03, 1.2253030e-04],
       [1.4992781e-02, 2.2020623e-02, 2.0736912e-02, ..., 1.1011103e-02,
        3.8169941e-03, 2.3434897e-05],
       ...,
       [5.2920748e-03, 9.8310029e-03, 1.7678655e-03, ..., 1.2859656e-02,
        1.8155276e-03, 3.6442585e-04],
       [7.2907668e-04, 1.8394399e-03, 3.2922224e-04, ..., 2.5836281e-03,
        8.9212852e-05, 2.9069197e-06],
       [7.7004419e-03, 3.6098685e-02, 3.9826348e-02, ..., 4.7233505e-03,
        2.4005175e-03, 2.0590196e-05]], dtype=float32)

In [276]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([ 8, 73, 32, ..., 48, 73, 68], dtype=int64)

In [277]:
true_labels = test_df['label']

In [278]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.15362873239568153

In [279]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.19462540716612378

In [280]:
# Evaluate the model on the test set
test_loss = model.evaluate(x=[X_test_pic, X_test_centaur, X_test_scale, X_test_nines, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 8s 100ms/step - loss: 3.4113 - accuracy: 0.2243
Test Loss: [3.411283016204834, 0.2243485301733017]


In [281]:
loaded_model = load_model('model_centaur_scale_nines_threes_dropout/')

In [282]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_centaur, X_test_scale, X_test_nines, X_test_threes])
test_predictions

77/77 [==============================] - 8s 98ms/step


array([[1.03340290e-05, 1.11056315e-05, 2.80437453e-05, ...,
        6.66835967e-06, 1.02657614e-05, 3.01877003e-06],
       [8.82584695e-03, 8.42931960e-03, 1.19575048e-02, ...,
        1.48952948e-02, 5.49513660e-03, 6.13257725e-05],
       [1.17831649e-02, 1.54574644e-02, 2.09136102e-02, ...,
        2.00052503e-02, 5.96382841e-03, 4.64930927e-05],
       ...,
       [1.34296743e-02, 1.46726687e-02, 1.02315238e-02, ...,
        3.58106084e-02, 5.29048918e-03, 1.38101241e-04],
       [1.17831649e-02, 1.54574644e-02, 2.09136102e-02, ...,
        2.00052503e-02, 5.96382841e-03, 4.64930927e-05],
       [5.56445494e-03, 2.45250706e-02, 3.55887748e-02, ...,
        5.35370363e-03, 3.26609425e-03, 3.16867954e-05]], dtype=float32)

In [283]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

array([ 8, 73, 32, ..., 80, 32, 86], dtype=int64)

In [284]:
true_labels = test_df['label']

In [285]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

0.10402570745862248

In [286]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

0.16286644951140064

In [288]:
# Evaluate the model on the test set
test_loss = model_dropout.evaluate(x=[X_test_pic, X_test_centaur, X_test_scale, X_test_nines, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

77/77 [==============================] - 6s 78ms/step - loss: 3.1189 - accuracy: 0.1629
Test Loss: [3.1189470291137695, 0.16286644339561462]


**CNN + nine_partition + three_partition + fitzpatrick_scale**

In [ ]:
df = pd.read_csv('fitzpatrick17k_3')
df

In [ ]:
df = df.drop(['fitzpatrick_centaur', 'qc', 'url'], axis=1)

In [ ]:
# Replace 'images' with 'images_resized' in the 'image_path' column
df['image_path'] = df['image_path'].str.replace('images_full', 'images_resized_full')

In [ ]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Visualize the unique and encoded labels
print("Unique labels:", label_encoder.classes_)
print("Encoded labels:", df['label'].unique())

In [ ]:
df = pd.get_dummies(df, columns=['fitzpatrick_scale'])

In [ ]:
npz_paths = []

# Loop over the DataFrame rows
for i, row in df.iterrows():
    # Get the image path and .npz file path
    picture_path = row['image_path']
    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    # Load the image
    pic_bgr_arr = cv2.imread(picture_path)
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    
    # Extracting Fitzpatrick scale, nine_partition, and three_partition labels
    minus_one_scale, ones_cale, two_scale = row['fitzpatrick_scale_-1'], row['fitzpatrick_scale_1'], row['fitzpatrick_scale_2']
    three_scale, four_scale, five_scale, six_scale = row['fitzpatrick_scale_3'], row['fitzpatrick_scale_4'], row['fitzpatrick_scale_5'], row['fitzpatrick_scale_6']
   
    ben_derm, ben_epiderm, ben_mel = row['nine_partition_label_benign dermal'], row['nine_partition_label_benign epidermal'], row['nine_partition_label_benign melanocyte']
    gen, inf = row['nine_partition_label_genodermatoses'], row['nine_partition_label_inflammatory']
    mal_cut, mal_der, mal_epider, mal_mel = row['nine_partition_label_malignant cutaneous lymphoma'], row['nine_partition_label_malignant dermal'], row['nine_partition_label_malignant epidermal'], row['nine_partition_label_malignant melanoma']
    
    ben, mal, non_neo = row['three_partition_label_benign'], row['three_partition_label_malignant'], row['three_partition_label_non-neoplastic']
    
    # Creating arrays for Fitzpatrick scale, nine_partition, and three_partition labels
    threes = np.array([ben, mal, non_neo])
    nines = np.array([ben_derm, ben_epiderm, ben_mel, gen, inf, mal_cut, mal_der, mal_epider, mal_mel])
    scale = np.array([minus_one_scale, ones_cale,  two_scale, three_scale, four_scale, five_scale, six_scale])
    
    
    # Getting the overall label
    label = row['label']

    # Save the data into the .npz file
    np.savez_compressed(npz_path, pic=pic_rgb_arr, scale=scale, nines=nines, threes=threes, label=label)

# Add the NPZ paths column to the DataFrame
df['NPZ_Path'] = pd.Series(npz_paths)

In [ ]:
scale_df = df[['fitzpatrick_scale_-1', 'fitzpatrick_scale_1', 'fitzpatrick_scale_2', 'fitzpatrick_scale_3', 'fitzpatrick_scale_4', 'fitzpatrick_scale_5', 'fitzpatrick_scale_6']]

In [ ]:
# Creating a list of features starting with 'nine_partition'
nine_features = [col for col in df.columns if col.startswith('nine_partition')]

# Creating a list of features starting with 'three_partition'
three_features = [col for col in df.columns if col.startswith('three_partition')]

# Creating the DataFrame nines_df with features starting with 'nine_partition'
nines_df = df[nine_features]

# Creating the DataFrame threes_df with features starting with 'three_partition'
threes_df = df[three_features]

In [ ]:
nines_df 

In [ ]:
threes_df 

In [ ]:
df = df.drop(columns=df.columns.difference(['image_name', 'label', 'NPZ_Path']))
df

In [ ]:
shuffled_df = df.sample(frac=1)

# Calculate the size of each set based on desired proportions
total_samples = len(shuffled_df)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

# Divide the DataFrame into training, validation, and test sets
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:train_size+val_size]
test_df = shuffled_df[train_size+val_size:]

len(train_df), len(val_df), len(test_df)

In [ ]:
def load_npz_data(npz_file):
    """Load data from a .npz file."""
    with np.load(npz_file, allow_pickle=True) as data:
        pic = data['pic']
        scale = data['scale']
        nines = data['nines']
        threes = data['threes']
        label = data['label']
    return pic, scale, nines, threes, label



def get_X_y(df):
    X_pic, X_scale, X_nines, X_threes= [], [], [], [], []
    y = []

    for name in df['NPZ_Path']:
        loaded_npz = np.load(name)

        pic = loaded_npz['pic']
        X_pic.append(pic)
        
        scale = loaded_npz['scale']
        X_scale.append(scale)
        
        nines = loaded_npz['nines']
        X_nines.append(nines)
        
        threes = loaded_npz['threes']
        X_threes.append(threes)

        y.append(loaded_npz['label'])

    X_pic, X_scale, X_nines, X_threes = np.array(X_pic), np.array(X_scale), np.array(X_nines), np.array(X_threes)
    y = np.array(y)

    return (X_pic, X_scale, X_nines, X_threes), y

(X_train_pic, X_train_scale, X_train_nines, X_train_threes), y_train = get_X_y(train_df)
(X_train_pic.shape, X_train_scale.shape, X_train_nines.shape, X_train_threes.shape), y_train.shape

In [ ]:
(X_val_pic, X_val_scale, X_val_nines, X_val_threes), y_val = get_X_y(val_df)
(X_val_pic.shape, X_val_scale.shape, X_val_nines.shape, X_val_threes.shape), y_val.shape

In [ ]:
(X_test_pic, X_test_scale, X_test_nines, X_test_threes), y_val = get_X_y(test_df)
(X_test_pic.shape, X_test_scale.shape, X_test_nines.shape, X_test_threes.shape), y_test.shape

In [ ]:
def build_model(conv_params, dense_params, learning_rate=0.0001, dropout=0):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(7,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output1 = y
    
     # Stats stream 2
    input_stats2 = layers.Input(shape=(9,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output2 = y
    
      # Stats stream 3
    input_stats3 = layers.Input(shape=(3,))  
    y = input_stats3
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output3 = y

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2, stats_output3])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2, input_stats3], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model
model = build_model(conv_params, dense_params)

In [ ]:
def build_model_dropout(conv_params, dense_params, learning_rate=0.0001, dropout=0.5):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each convolutional layer
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each dense layer
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(7,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output1 = y
    
    # Stats stream 2
    input_stats2 = layers.Input(shape=(9,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output2 = y 
    
    # Stats stream 3
    input_stats3 = layers.Input(shape=(3,))  
    y = input_stats3
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output3 = y 
    

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2, stats_output3])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2, input_stats3], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model with dropout rate of 0.5
model_dropout = build_model_dropout(conv_params, dense_params, dropout=0.5)

In [ ]:
model.summary()

In [ ]:
model_dropout

In [ ]:
cp = ModelCheckpoint('model_scale_nines_threes/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_scale, X_train_nines, X_train_threes], y=y_train,
          validation_data=([X_val_pic, X_val_scale, X_val_nines, X_val_threes], y_val),
          epochs=20, 
          callbacks=[cp])

In [ ]:
cp = ModelCheckpoint('model_scale_nines_threes_dropout/', save_best_only=True)

model_dropout.fit(x=[X_train_pic, X_train_scale, X_train_nines, X_train_threes], y=y_train,
          validation_data=([X_val_pic, X_val_scale, X_val_nines, X_val_threes], y_val),
          epochs=20, 
          callbacks=[cp])

In [ ]:
loaded_model = load_model('model_scale_nines_threes/')

In [ ]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_scale, X_test_nines, X_test_threes])
test_predictions

In [ ]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

In [ ]:
true_labels = test_df['label']

In [ ]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

In [ ]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

In [ ]:
# Evaluate the model on the test set
test_loss = model.evaluate(x=[X_test_pic, X_test_centaur, X_test_scale, X_test_nines, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

In [ ]:
loaded_model = load_model('model_scale_nines_threes_dropout/')

In [ ]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_scale, X_test_nines, X_test_threes])
test_predictions

In [ ]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

In [ ]:
true_labels = test_df['label']

In [ ]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

In [ ]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

In [ ]:
# Evaluate the model on the test set
test_loss = model_dropout.evaluate(x=[X_test_pic, X_test_centaur, X_test_scale, X_test_nines, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

**CNN + fitzpatrick_scale + nine_partition**

In [ ]:
df = pd.read_csv('fitzpatrick17k_3')
df

In [ ]:
df = df.drop(['fitzpatrick_centaur', 'qc', 'url', 'three_partition_label_benign', 'three_partition_label_malignant',
              'three_partition_label_non-neoplastic'], axis=1)

In [ ]:
# Replace 'images' with 'images_resized' in the 'image_path' column
df['image_path'] = df['image_path'].str.replace('images_full', 'images_resized_full')

In [ ]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
df['label'] = label_encoder.fit_transform(df['label'])

# Visualize the unique and encoded labels
print("Unique labels:", label_encoder.classes_)
print("Encoded labels:", df['label'].unique())

In [ ]:
df = pd.get_dummies(df, columns=['fitzpatrick_scale'])

In [ ]:
npz_paths = []

# Loop over the DataFrame rows
for i, row in df.iterrows():
    # Get the image path and .npz file path
    picture_path = row['image_path']
    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    # Load the image
    pic_bgr_arr = cv2.imread(picture_path)
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    
    # Extracting Fitzpatrick scale, nine_partition, and three_partition labels
    minus_one_scale, ones_cale, two_scale = row['fitzpatrick_scale_-1'], row['fitzpatrick_scale_1'], row['fitzpatrick_scale_2']
    three_scale, four_scale, five_scale, six_scale = row['fitzpatrick_scale_3'], row['fitzpatrick_scale_4'], row['fitzpatrick_scale_5'], row['fitzpatrick_scale_6']
   
    ben_derm, ben_epiderm, ben_mel = row['nine_partition_label_benign dermal'], row['nine_partition_label_benign epidermal'], row['nine_partition_label_benign melanocyte']
    gen, inf = row['nine_partition_label_genodermatoses'], row['nine_partition_label_inflammatory']
    mal_cut, mal_der, mal_epider, mal_mel = row['nine_partition_label_malignant cutaneous lymphoma'], row['nine_partition_label_malignant dermal'], row['nine_partition_label_malignant epidermal'], row['nine_partition_label_malignant melanoma']
    
    
    # Creating arrays for Fitzpatrick scale, nine_partition
    nines = np.array([ben_derm, ben_epiderm, ben_mel, gen, inf, mal_cut, mal_der, mal_epider, mal_mel])
    scale = np.array([minus_one_scale, ones_cale,  two_scale, three_scale, four_scale, five_scale, six_scale])
    
    
    # Getting the overall label
    label = row['label']

    # Save the data into the .npz file
    np.savez_compressed(npz_path, pic=pic_rgb_arr, scale=scale, nines=nines, label=label)

# Add the NPZ paths column to the DataFrame
df['NPZ_Path'] = pd.Series(npz_paths)

In [ ]:
scale_df = df[['fitzpatrick_scale_-1', 'fitzpatrick_scale_1', 'fitzpatrick_scale_2', 'fitzpatrick_scale_3', 'fitzpatrick_scale_4', 'fitzpatrick_scale_5', 'fitzpatrick_scale_6']]

In [ ]:
# Creating a list of features starting with 'nine_partition'
nine_features = [col for col in df.columns if col.startswith('nine_partition')]
# Creating the DataFrame nines_df with features starting with 'nine_partition'
nines_df = df[nine_features]

nines_df

In [ ]:
df = df.drop(columns=df.columns.difference(['image_name', 'label', 'NPZ_Path']))
df

In [ ]:
shuffled_df = df.sample(frac=1)

# Calculate the size of each set based on desired proportions
total_samples = len(shuffled_df)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)

# Divide the DataFrame into training, validation, and test sets
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:train_size+val_size]
test_df = shuffled_df[train_size+val_size:]

len(train_df), len(val_df), len(test_df)

In [ ]:
def load_npz_data(npz_file):
    """Load data from a .npz file."""
    with np.load(npz_file, allow_pickle=True) as data:
        pic = data['pic']
        scale = data['scale']
        nines = data['nines']
        label = data['label']
    return pic, scale, nines, label



def get_X_y(df):
    X_pic, X_scale, X_nines, = [], [], [], []
    y = []

    for name in df['NPZ_Path']:
        loaded_npz = np.load(name)

        pic = loaded_npz['pic']
        X_pic.append(pic)
        
        scale = loaded_npz['scale']
        X_scale.append(scale)
        
        nines = loaded_npz['nines']
        X_nines.append(nines)

        y.append(loaded_npz['label'])

    X_pic, X_scale, X_nines = np.array(X_pic), np.array(X_scale), np.array(X_nines)
    y = np.array(y)

    return (X_pic, X_scale, X_nines), y

(X_train_pic, X_train_scale, X_train_nines), y_train = get_X_y(train_df)
(X_train_pic.shape, X_train_scale.shape, X_train_nines.shape), y_train.shape

In [ ]:
(X_val_pic, X_val_scale, X_val_nines), y_val = get_X_y(val_df)
(X_val_pic.shape, X_val_scale.shape, X_val_nines.shape), y_val.shape

In [ ]:
(X_test_pic, X_test_scale, X_test_nines), y_test = get_X_y(test_df)
(X_test_pic.shape, X_test_scale.shape, X_test_nines.shape), y_test.shape

In [ ]:
def build_model(conv_params, dense_params, learning_rate=0.0001, dropout=0):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(7,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output1 = y
    
     # Stats stream 2
    input_stats2 = layers.Input(shape=(9,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
    stats_output2 = y
    

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model
model = build_model(conv_params, dense_params)

In [ ]:
def build_model_dropout(conv_params, dense_params, learning_rate=0.0001, dropout=0.5):
    """
    Parameters:
        conv_params (tuple): Tuple containing information about convolutional layers.
                             Format: (n_conv2d, filters_conv2d)
            n_conv2d (int): Number of convolutional layers.
            filters_conv2d (tuple): Tuple of number of filters for each convolutional layer.
        dense_params (tuple): Tuple containing information about dense layers.
                              Format: (n_dense, units_dense)
            n_dense (int): Number of dense layers.
            units_dense (list): List of number of units/neurons for each dense layer.
        dropout (float): Dropout rate.

    Returns:
        model: Neural network model.
    """
    num_classes = 114

    # Picture (CNN) stream 
    input_pic = layers.Input(shape=(96, 96, 3))
    x = input_pic
    for i in range(conv_params[0]):
        x = layers.Conv2D(conv_params[1][i], (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each convolutional layer
    x = layers.Flatten()(x)
    for units in dense_params[1]:
        x = layers.Dense(units, activation='relu')(x)
        x = Dropout(dropout)(x)  # Dropout layer with dropout rate of 0.5 after each dense layer
    pic_output = x

    
    # Stats stream 1
    input_stats1 = layers.Input(shape=(7,))  
    y = input_stats1
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output1 = y
    
    # Stats stream 2
    input_stats2 = layers.Input(shape=(9,))  
    y = input_stats2
    for units in dense_params[1]:
        y = layers.Dense(units, activation='relu')(y)
        y = Dropout(dropout)(y)  # Dropout layer with dropout rate of 0.5 after each dense layer
    stats_output2 = y 
    

    # Concatenate the outputs of both streams
    combined = layers.concatenate([pic_output, stats_output1, stats_output2])

    # Output layer
    z = layers.Dense(num_classes, activation='softmax')(combined)

    # Define the complete model
    model = Model(inputs=[input_pic, input_stats1, input_stats2], outputs=z)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define parameters for convolutional and dense layers
conv_params = (3, (32, 64, 128))  # Three convolutional layers with 32, 64, and 128 filters
dense_params = (1, [114])  # One dense layer with 114 units

# Build the model with dropout rate of 0.5
model_dropout = build_model_dropout(conv_params, dense_params, dropout=0.5)

In [ ]:
model.summary()

In [ ]:
model_dropout.summary()

In [ ]:
cp = ModelCheckpoint('model_scale_nines/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_scale, X_train_nines], y=y_train,
          validation_data=([X_val_pic, X_val_scale, X_val_nines], y_val),
          epochs=20, 
          callbacks=[cp])

In [ ]:
cp = ModelCheckpoint('model_scale_nines_dropout/', save_best_only=True)

model_dropout.fit(x=[X_train_pic, X_train_scale, X_train_nines], y=y_train,
          validation_data=([X_val_pic, X_val_scale, X_val_nines], y_val),
          epochs=20, 
          callbacks=[cp])

In [ ]:
loaded_model = load_model('model_scale_nines/')

In [ ]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_scale, X_test_nines])
test_predictions

In [ ]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

In [ ]:
true_labels = test_df['label']

In [ ]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

In [ ]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

In [ ]:
# Evaluate the model on the test set
test_loss = model.evaluate(x=[X_test_pic, X_test_centaur, X_test_scale, X_test_nines, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)

In [ ]:
loaded_model = load_model('model_scale_nines_dropout/')

In [ ]:
# Making predictions
test_predictions = loaded_model.predict([X_test_pic, X_test_scale, X_test_nines])
test_predictions

In [ ]:
# Getting the class corresponding to the highest probability for each row
predicted_classes = np.argmax(test_predictions, axis=1)
predicted_classes

In [ ]:
true_labels = test_df['label']

In [ ]:
f1 = f1_score(true_labels, predicted_classes, average='weighted')
f1

In [ ]:
accuracy = accuracy_score(true_labels, predicted_classes)
accuracy

In [ ]:
# Evaluate the model on the test set
test_loss = model_dropout.evaluate(x=[X_test_pic, X_test_centaur, X_test_scale, X_test_nines, X_test_threes], y=y_test)

# Print the test loss
print("Test Loss:", test_loss)